In [1]:
#versione del 11/12/2018 con (modifiche in ordine cronologico):
#- AnswerDuration rettificata con lo storno delle API Call dal BotAnswerDuration
#- nuovo campo "EventHourMinute"
#- 29/11/2018: nuovo codice per i Dataframe df_bot_messages e df_user_message
#- 01/12/2018: riorganizzazione delle scritture (prima DW e poi delta Table) per evitare i duplicati e update a type = "previous" dei record dopo la scrittura in Delta Table
#- 01/12/2018: delete dalla Delta Table "Events" per i record dello slot orario che si sta processando, così non ci sono problemi di duplicati in caso di riprocessamento
#- 03/12/2018: mapping del campo "envionment" nel flusso fino alla Delta Table "events" e al Datawarehouse
#- 05/12/2018: TXTAnalytics cast from string to float and DF filtered out from EventName != ServiceProfilerSample
#- 05/12/2018: added new field "EventNameDecode"
#- 06/12/2018: added df_5_final.persist()
#- 11/12/2018: commentati i persist/unpersist, valorizzato a false spark.databricks.queryWatchdog.enabled , spostata la scrittura sulla delta table prima della scrittura su DWH, inserito filtro sulle lettura della Delta EventsMirror (source) per non processare la data ed ora del RUN

#VERSIONE DEL 19/12/2018: corretto l'errore per cui ConversationStartDate, ConversationStartTime e ConversationEndTime avevano Null. Ora vengono valorizzati correttamente

In [2]:
%run "./Include/ETL_Main_Setup"

In [3]:
dbutils.widgets.text("input", "","")
dbutils.widgets.get("input")
datetimefolders = getArgument("input")
print(datetimefolders)
# prova 2018-11-12/11  per il Windowing, 2018-11-12/14 per bot answerduration. Oppure 2018-12-13/Federico con Pippo e ALDO. o i 3 files vecchi in Federico_old

2018-11-12/14

In [4]:
spark.conf.set("dfs.adls.oauth2.access.token.provider.type", "ClientCredential")
spark.conf.set("dfs.adls.oauth2.client.id", "c9463783-977a-4fd6-be5e-685771eee8da")
spark.conf.set("dfs.adls.oauth2.credential", "bUB7Yo1o7M0e+yU0eNkT44cmRARwtjXSAGc7mR2qP9c= ")
spark.conf.set("dfs.adls.oauth2.refresh.url", "https://login.microsoftonline.com/6815f468-021c-48f2-a6b2-d65c8e979dfb/oauth2/token")

databasetablePath = "adl://timvadevdls.azuredatalakestore.net/databricks/"

In [5]:
spark.conf.set("spark.databricks.delta.formatCheck.enabled", "false")

mount_storage(storage_account_name,storage_account_access_key)

In [6]:
%sql 
SELECT *
FROM DEV.Events

itemId,ClientAtHome,LUISintent,LUISscore,QNAanswer,QNAcategory,QNAscore,QNAsource,TXTANALYTICSscore,activityId,channel,conversationId,currentScenario,diagnosticResponse,dialogCloseType,endTimeAPICall,eventName,eventTime,eventDate,isLeaf,mainScenario,modemAlignment,modemChanged,modemType,reason,scenario,scenarioCalled,scenarioId,startTimeAPICall,telemetryId,text,ticket,ticketForm,timestamp,toolExec,toolResult,userId,userName,type,apiTimeDiff,BotAnswerDuration,QNAuserMessage,EventTimePrevious,notUnderstoodMessage,LUISentities,DateUTCInsertRow,applicationInsight,businessUnit,ProblemSolved,user,isAuthenticated,Status,Recognition,StatusCode,RecognitionCode,HourFromLastEvent,EventHourMinute,Environment,eventNameDecode


In [7]:
%sql 
SELECT count(*)
FROM DEV.Events

count(1)
0


In [9]:
%sql 
SELECT count(*)
FROM DEV.Conversations

count(1)
131


In [10]:
dtf = "2018-10-02/"
hh = list(range(0,23))

#path = file_location + datetimefolders +  "/*.blob"
parser_wo = AppInsightEventParser(storage_account_name,container_name_wo,folder_name_wo)
parser_bu = AppInsightEventParser(storage_account_name,container_name_bu,folder_name_bu)

parser_wo.addStage(Stage1())
parser_bu.addStage(Stage1())
#parser_wo.addStage(Stage2("/delta/DEV/events/"))
#parser_wo.addStage(Stage3())
#parser_wo.addStage(Stage4())

#dw_info_wo = DWInfo(dwh_server,dwh_db_name,"timsqladmin@tim-va-dev-sqlserver","P@ssword!")
#parser_wo.addStage(Stage5(dw_info_wo))


#datetimefolders = dtf + str(hh)
df_wo = parser_wo.parseBlob(datetimefolders,parseRow)
#df_wo.printSchema()
#display(df_wo)
df_bu = parser_bu.parseBlob(datetimefolders,parseRow)
#df_bu.printSchema()
#display(df_bu)
#df_co = parser_co.parseBlob(datetimefolders,parseRow)
#display(df_bu)
# 2018-11-19 Simone Begin ->
df_wo = df_wo.withColumn("applicationInsight", F.lit(container_name_wo))
df_wo = df_wo.withColumn("businessUnit", F.lit(businessUnitName_wo))
df_bu = df_bu.withColumn("applicationInsight", F.lit(container_name_bu))
df_bu = df_bu.withColumn("businessUnit", F.lit(businessUnitName_bu))
#df_co = df_co.withColumn("applicationInsight", F.lit(container_name_co))
#df_co = df_co.withColumn("businessUnit", F.lit(businessUnitName_co))

""" AVOIDING PROBLEMS OF SCHEMA """
if df_wo.count()==0 and df_bu.count()> 0 :
    print('df_wo is empty')
    df_total= df_bu
elif df_bu.count()==0 and df_wo.count()> 0:
    print('df_bu is empty')
    df_total= df_wo
elif df_bu.count()>0 and df_wo.count()> 0:
    print("both wo and bu have records")
    df_total= df_wo.union(df_bu)
else:
    print("non c'è nessuno dei due path")

"""df_total = df_wo.union(df_bu)  """  # 3/12/2018 Federico: it was --> df_bu.union(df_wo)

# Union of previous united WO and BU dataframes with CO dataframe

#df_total = df_total.union(df_co)

"""display(df_total)"""
# 2018-11-19 Simone -> End
#df_total.printSchema()
df_1 = df_total
cols_pilot = df_1.columns
df_1 = df_1.withColumn("type", F.lit("new"))

#df.createOrReplaceTempView("temp")
#df.write.mode("append").format("delta").partitionBy("eventName").save("/mnt/delta/eventsdl")

#spark.sql("OPTIMIZE eventsdl ZORDER BY (conversationId)")

#display(df_1)

root
-- ClientAtHome: string (nullable = true)
-- LUISentities: string (nullable = true)
-- LUISintent: string (nullable = true)
-- LUISscore: string (nullable = true)
-- QNAanswer: string (nullable = true)
-- QNAcategory: string (nullable = true)
-- QNAscore: string (nullable = true)
-- QNAsource: string (nullable = true)
-- QNAuserMessage: string (nullable = true)
-- TXTANALYTICSscore: string (nullable = true)
-- activityId: string (nullable = true)
-- channel: string (nullable = true)
-- conversationId: string (nullable = true)
-- currentScenario: string (nullable = true)
-- diagnosticResponse: string (nullable = true)
-- dialogCloseType: string (nullable = true)
-- endTimeAPICall: string (nullable = true)
-- eventName: string (nullable = true)
-- eventTime: string (nullable = true)
-- eventdate: string (nullable = true)
-- isAuthenticated: string (nullable = true)
-- isLeaf: string (nullable = true)
-- itemId: string (nullable = true)
-- mainScenario: string (nullable = true)
-- modemAlignment: string (nullable = true)
-- modemChanged: string (nullable = true)
-- modemType: string (nullable = true)
-- notUnderstoodMessage: string (nullable = true)
-- problemSolved: string (nullable = true)
-- reason: string (nullable = true)
-- scenario: string (nullable = true)
-- scenarioCalled: string (nullable = true)
-- scenarioId: string (nullable = true)
-- startTimeAPICall: string (nullable = true)
-- telemetryId: string (nullable = true)
-- text: string (nullable = true)
-- ticket: string (nullable = true)
-- ticketForm: string (nullable = true)
-- timestamp: string (nullable = true)
-- toolExec: string (nullable = true)
-- toolResult: string (nullable = true)
-- user: string (nullable = true)
-- userId: string (nullable = true)
-- userName: string (nullable = true)
-- applicationInsight: string (nullable = true)
-- businessUnit: string (nullable = true)

df_bu is empty

In [11]:
#%fs rm -r mnt/delta/eventsdl

In [12]:
df_1= df_1.withColumn('TXTANALYTICSscore' , df_1['TXTANALYTICSscore'].cast(FloatType()))
df_1= df_1.select(['*']).where(col('EventName')!= 'ServiceProfilerSample')

In [ ]:
df_1= df_1.withColumn('eventName', when(df_1.conversationId== 'ALDO', F.lit('MBFEvent.QNAEvent')).otherwise(df_1.eventName)).withColumn('QNAcategory', 
                                             when(df_1.conversationId == 'ALDO', F.lit('ChitChat')).otherwise(df_1.QNAcategory)) #modificata ad hoc per far sì che venga in corso
#display(df_1) #4 record nello slot orario corrente

In [14]:
df_1.count()  #921 perchè Events è vuota

Out[ 141 ]: 921

In [15]:
#tableName = "adl://timvadevdls.azuredatalakestore.net/delta/eventsdl/"

#spark.sql("OPTIMIZE '/mnt/delta/eventsdl' ZORDER BY (conversationId)")

In [16]:
#spark.read.format("delta").load(tableName).count()
#ccdf = spark.sql("SELECT * FROM delta.`/mnt/delta/eventsdl` WHERE eventName = 'MBFEvent.CallScenario' AND conversationId = 'BTvzSeL1ZzC7b1MZDplwoK'")

#display(ccdf)

In [17]:
def parseLuisEntity(x):
    
    rowDict = x.asDict()
    jsonList = rowDict.pop('LUISentities')

    js_d = json.loads(jsonList)
    for entity in js_d:
        rowDict['LuisEntityKey'] = entity["key"]
        rowDict['LuisEntityValue'] = entity["value"]
        rowDict['LuisEntityScore'] = entity["score"]
        yield Row(**rowDict)

                          
def timestamp_to_long(dt):
    """
    This function takes a timestamp and returns it as microseconds from epoc, respecting timezone.  If no tzinfo is specified,
    UTC is assumed.
    :param dt:
    :return:
    """
    if dt is None:
        return None

    if not dt.tzinfo:
        dt = dt.replace(tzinfo=datetime.timezone.utc)

    return int(
        ((dt - datetime.datetime(1970, 1, 1, tzinfo=datetime.timezone.utc)) / datetime.timedelta(microseconds=1)))


def timestamp_diff(startTime, endTime):
  
    if startTime is None or endTime is None:
        return 0
  
    tt = timestamp_to_long(endTime) - timestamp_to_long(startTime)
    return tt / 1000

In [18]:
#ENTITIES MANAGEMENT
dfEntities = df_1.where((df_1["LUISentities"] != '') & (df_1["LUISentities"] != '[]')).select("conversationId","timestamp","itemId","LUISentities")
#flatMapping
mappedj = dfEntities.rdd.flatMap(parseLuisEntity)

#If entities have been found, then write them in the DWH
if(mappedj.isEmpty() == False):
    # Create a DataFrame from the mapped collection
    df_forEntitiesTable = spark.createDataFrame(mappedj)
    df_forEntitiesTable = df_forEntitiesTable.withColumn("DateUTCInsertRow", from_utc_timestamp(F.lit(current_timestamp()),"Europe/Rome")) #Andrea 25/10/2018
    df_forEntitiesTable = df_forEntitiesTable.select("conversationId","itemId","timestamp","LuisEntityKey","LuisEntityValue","LuisEntityScore","DateUTCInsertRow")
    # Entities written in dedicated table into the Datawarehouse: [EventEntity] table
    df_forEntitiesTable.write.option("maxStrLength",4000).format("com.databricks.spark.sqldw").option("url", connStringDWH).option("dbtable", "EventEntity").option("forward_spark_azure_storage_credentials","True").mode("append").option("tempdir", tempdatabricks_wo).save()
    else:
        print(mappedj.isEmpty())

In [19]:
print(datetimefolders)
datefolder = datetimefolders[0:10]
print(datefolder)
datehour = datetimefolders[11:13]
print(datehour)

2018-11-12/14
2018-11-12
14

In [20]:
#BEFORE READ, DELETE FROM DELTA TABLE
#spark.sql(" DELETE FROM delta.`/delta/{0}/Events/` WHERE EventDate||'/'||hour(EventTime) = '{1}' ".format(environment, datetimefolders))  
  
# Reading data from Delta Table (DBFS) to take data from the past
previous_events = spark.read.format("delta").load("/delta/"+environment+"/Events/").select(cols_pilot)

#il giorno 2018-12-13 ho commentato parte sotto perchè per qualche motivo non importava alcuna riga dalla delta, ovvero previous events è vuota
"""
#MODIFIED ON 2018-12-11 adding logic for filtering duplicates, not based on deleting infos, but filtering them and at the overwriting them at the end of the present notebook.
previous_events_filtered = spark.sql("SELECT * FROM {0}.Events WHERE eventDate != '{1}' AND hour(eventTime) != '{2}' ".format(environment, datefolder, datehour))
previous_events = previous_events_filtered.select(cols_pilot)
#previous_events_filtered.printSchema()
#previous_events = spark.sql("SELECT eventDate, hour(eventTime) as Hour FROM previous_events_table GROUP BY eventDate, hour(eventTime) ORDER BY eventDate, hour(eventTime)")"""
                                     #WHERE eventDate = '{1}' AND hour(EventTime) = '{2}' ".format(environment, datefolder, datehour))"""
#display(previous_events)
previous_events = previous_events.withColumn("type", F.lit("previous"))
# Union of two DataFrames
unionDataFrame = previous_events.unionAll(df_1)

In [21]:
previous_events.count()  #ho 8 record nella delta

Out[ 146 ]: 0

In [22]:
unionDataFrame.count()  #ha 8+4=12 records

Out[ 147 ]: 921

In [ ]:
#insert apiTimeDiff calculated column only when I have Event= ApiCall, otherwise leaving null (default)
func_udf = udf(timestamp_diff, FloatType())
df_allEvents = unionDataFrame.withColumn("apiTimeDiff", when(unionDataFrame.eventName == 'MBFEvent.APICall', func_udf(unionDataFrame["startTimeAPICall"], unionDataFrame["endTimeAPICall"])))
df_allEvents.createOrReplaceTempView('allEvents')
display(df_allEvents)

In [24]:
df_allEvents.count()

Out[ 149 ]: 921

In [25]:
# Spiegazione Windowing
"""Per ogni ActivityId di ogni conversationId voglio calcolare la botAnswerDuration, intesa come l'intervallo di tempo che intercorre tra il primo messaggio dell'utente e la prima risposta del Bot. Per cui Per ogni activityId prendo questi due eventi, MBFEvent.UserMessage e MBFEvent.BotMessage, dove il BotMessage deve essere in un eventTime successivo, e avrò una situazione del tipo:
ActivityId    EventName    EventTime
   A           UserMessage    15:30
   A           BotMessage     15:35
 Adesso uso il Windowing per 'spostare' l eventTime del record dell UserMessage sulla riga successiva, in modo che poi posso fare la sottrazione. ovvero avrò:
 ActivityId    EventName    EventTime  EventTimePrevious   BotAnswerDuration
   A           UserMessage    15:30         /                    /
   A           BotMessage     15:35         15:35               00:05
   
   L'ho 'trascinato' sotto, quindi Windowing crea una finestra di un record, nel senso che non guardo solo il un record singolo ma anche quello prima. Avendoli ora sulla stessa riga, con un semplice withColumn posso calcolare BotAnswerDuration facendo la sottrazione.
   Infine vado in join left con Events, perchè questo Windowing è fatto su un sottoinsieme di tutti i record, ovvero solo quelli corrispondenti agli eventi UserMessage e BotMessage (Es: Se Events ha 1000 record, Windowing lavora su 150. Facendo Events.join(df_Windowing, how='left') riottengo tutti i 1000 record con il nuovo campo botAnswerDuration che sarà valorizzato solo su alcuni dei 150 (sugli eventi BotMessage dei 150))."""

In [26]:
#DATAFRAME with ONLY userMessages and botMessage for calculating responseTime
df_bot_messages = spark.sql(" \
          SELECT A.ConversationId, A.ActivityId, A.eventName, A.itemId,  A.eventTime \
          FROM allEvents A \
          INNER JOIN ( \
                            SELECT AA.itemId, AA.ActivityId \
                            FROM allEvents AA \
                            INNER JOIN ( \
                                        SELECT ActivityId ,min(eventTime) as eventTime \
                                        FROM allEvents \
                                        WHERE eventName = 'MBFEvent.BotMessage' and ActivityId is not null \
                                        GROUP BY ActivityId \
                                    ) BB \
                              on AA.ActivityId = BB.ActivityId and AA.eventTime = BB.eventTime \
                            GROUP BY AA.itemId, AA.ActivityId \
                    ) B \
           ON A.ActivityId = B.ActivityId and A.itemId = B.itemId \
           WHERE A.eventName = 'MBFEvent.BotMessage' \
          ")

df_user_messages = spark.sql(" \
          SELECT A.ConversationId, A.ActivityId, A.eventName, A.itemId, B.eventTime \
          FROM allEvents as A \
          INNER JOIN ( \
                      SELECT itemId, ActivityId ,min(eventTime) as eventTime \
                      FROM allEvents B \
                       WHERE eventName = 'MBFEvent.UserMessage' and ActivityId is not null \
                      GROUP BY itemId, ActivityId \
                  ) B \
          ON A.itemId = B.itemId \
	       ")

df_all_messages = df_bot_messages.union(df_user_messages).orderBy('ConversationId', 'activityId', 'eventTime', 'itemId')

# TimeDiff between UserMessage and BotMessage and insert botTimeDuration in df_2_time new dataframe
df_all_messages = df_all_messages.select("ConversationId","ActivityId","itemId","eventName","eventTime").orderBy("conversationid","eventTime")
display(df_all_messages)

ConversationId,ActivityId,itemId,eventName,eventTime
2cpS4QCdLps7ycDeAxodOz,b71e70ef-775c-4402-aac9-1c648ab74fef,056046af-e683-11e8-8c74-1df3eaf4721b,MBFEvent.BotMessage,2018-11-12T13:58:10.061+0000
2cpS4QCdLps7ycDeAxodOz,2cpS4QCdLps7ycDeAxodOz|0000046,1773f620-e683-11e8-8c74-1df3eaf4721b,MBFEvent.UserMessage,2018-11-12T13:58:27.503+0000
2cpS4QCdLps7ycDeAxodOz,2cpS4QCdLps7ycDeAxodOz|0000046,1773f627-e683-11e8-8c74-1df3eaf4721b,MBFEvent.BotMessage,2018-11-12T13:58:28.363+0000
2cpS4QCdLps7ycDeAxodOz,2cpS4QCdLps7ycDeAxodOz|0000050,1773f628-e683-11e8-8c74-1df3eaf4721b,MBFEvent.UserMessage,2018-11-12T13:58:30.353+0000
2cpS4QCdLps7ycDeAxodOz,2cpS4QCdLps7ycDeAxodOz|0000050,1773f62e-e683-11e8-8c74-1df3eaf4721b,MBFEvent.BotMessage,2018-11-12T13:58:30.494+0000
2cpS4QCdLps7ycDeAxodOz,2cpS4QCdLps7ycDeAxodOz|0000054,1773f632-e683-11e8-8c74-1df3eaf4721b,MBFEvent.UserMessage,2018-11-12T13:58:34.072+0000
2cpS4QCdLps7ycDeAxodOz,2cpS4QCdLps7ycDeAxodOz|0000054,1773f637-e683-11e8-8c74-1df3eaf4721b,MBFEvent.BotMessage,2018-11-12T13:58:34.337+0000
2cpS4QCdLps7ycDeAxodOz,af273cfd-1112-48af-a5a7-f760946bfe07,1773f63a-e683-11e8-8c74-1df3eaf4721b,MBFEvent.BotMessage,2018-11-12T13:58:44.846+0000
2cpS4QCdLps7ycDeAxodOz,2cpS4QCdLps7ycDeAxodOz|0000059,1773f641-e683-11e8-8c74-1df3eaf4721b,MBFEvent.UserMessage,2018-11-12T13:58:49.947+0000
2cpS4QCdLps7ycDeAxodOz,2cpS4QCdLps7ycDeAxodOz|0000059,1773f643-e683-11e8-8c74-1df3eaf4721b,MBFEvent.BotMessage,2018-11-12T13:58:49.960+0000


In [ ]:
time_window = Window.partitionBy("activityId").orderBy("ActivityId", "eventTime")
df_all_messages = df_all_messages.withColumn("prev_time", F.lag(df_all_messages.eventTime).over(time_window))
df_all_messages = df_all_messages.withColumn("eventTime", df_all_messages["eventTime"].cast(TimestampType()))
df_all_messages = df_all_messages.withColumn("prev_time", df_all_messages["prev_time"].cast(TimestampType()))
df_all_messages = df_all_messages.withColumn("BotAnswerDuration", func_udf(df_all_messages["prev_time"], df_all_messages["eventTime"]))
# df_all_messages = df_all_messages.withColumn("BotAnswerDuration", func_udf(df_all_messages["BotAnswerDuration"], df_all_messages["apiTimeDiff"]))
df_all_messages = df_all_messages.withColumn("EventTimePrevious", to_timestamp(df_all_messages["prev_time"]) ) #add column with Date of Previous Message, to do mapping of field also in DW
#display(df_all_messages).groupby('ConversationId', 'ActivityId').select(['*'])
df_with_BotTimeDuration = df_all_messages.select('itemId', 'BotAnswerDuration', 'EventTimePrevious').where(col("BotAnswerDuration").isNotNull())

#insert BotTimeDuration to a new dataframe
df_2_time = df_allEvents.join(df_with_BotTimeDuration, 'itemId', 'left')
display(df_2_time)
#display(df_2_time.filter(df_2_time.activityId == 'JEBXXIfay3m328mWQLvL0x|0000160').select('eventName', 'eventTime', 'BotAnswerDuration', 'apiTimeDiff', 'EventTimePrevious'))

In [28]:
#PREPARE DATA TO BE WRITTEN ENRICHED DATA INTO DWH Tables and STORE it in Databricks TABLES
df_3_final = df_2_time
df_3_final = df_3_final.withColumn('eventdate', df_3_final['eventTime'].cast('string').substr(1,10)) #cast from date to string and take left 10 characters to have a string in format "yyyy-mm-dd"
df_3_final = df_3_final.withColumn("DateUTCInsertRow", from_utc_timestamp(F.lit(current_timestamp()),"Europe/Rome"))


#2018-11-05 Simone/Federico
from types import ClassType
ClassType(1)
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType, NullType
udf_to_float = UserDefinedFunction(lambda x: float(0) if ClassType(x) != float else x, FloatType() )    
df_4_final = df_3_final.select(*[udf_to_float(column).alias('BotAnswerDuration') if column == 'BotAnswerDuration' else column for column in df_3_final.columns])
"""METTERE QUESTA SOTTO (df_5_final) SOLAMENTE SE VOGLIO CAMBIARE I NULL DI APITIMEDIFF CON DEGLI ZERO"""
#df_5_final = df_4_final.select(*[udf_to_float(column).alias('apiTimeDiff') if column == 'apiTimeDiff' else column for column in df_4_final.columns])#
#2018-11-05 Simone/Federico    

Out[ 152 ]: 'METTERE QUESTA SOTTO (df_5_final) SOLAMENTE SE VOGLIO CAMBIARE I NULL DI APITIMEDIFF CON DEGLI ZERO'

In [29]:
df_2_time.count()   #vengono creati ERRONAMENTE PIù RECORD col windowing. Debuggare

Out[ 153 ]: 921

###### Selection of records in df_4_final having BotAnswerDuration > 0

In [31]:
df_4_final.count()

Out[ 154 ]: 921

In [32]:
df_4_final.createOrReplaceTempView('table_4_final')

#df_5_final.printSchema()

df_bot_responseTime = spark.sql(" \
          SELECT ConversationId as botConversationId, ActivityId as botActivityId, eventName as botEventName, BotAnswerDuration as botBotAnswerDuration, (apiTimeDiff) as botApiTimeDiff, eventTime as botEventTime, startTimeAPICall as botStartTimeAPICall, endTimeAPICall as botEndTimeAPICall  \
          FROM table_4_final \
          WHERE BotAnswerDuration > 0 AND eventName = 'MBFEvent.BotMessage' \
          GROUP BY ConversationId, ActivityId, eventName, BotAnswerDuration, apiTimeDiff, eventTime, startTimeAPICall, endTimeAPICall ORDER BY eventTime  \
          ")
display(df_bot_responseTime)

botConversationId,botActivityId,botEventName,botBotAnswerDuration,botApiTimeDiff,botEventTime,botStartTimeAPICall,botEndTimeAPICall
777Qsjs3i8qJyBr95zABbk,777Qsjs3i8qJyBr95zABbk|0000050,MBFEvent.BotMessage,640.782,null,2018-11-12T13:58:00.722+0000,null,null
777Qsjs3i8qJyBr95zABbk,777Qsjs3i8qJyBr95zABbk|0000057,MBFEvent.BotMessage,528.878,null,2018-11-12T13:58:15.443+0000,null,null
2cpS4QCdLps7ycDeAxodOz,2cpS4QCdLps7ycDeAxodOz|0000046,MBFEvent.BotMessage,859.797,null,2018-11-12T13:58:28.363+0000,null,null
2cpS4QCdLps7ycDeAxodOz,2cpS4QCdLps7ycDeAxodOz|0000050,MBFEvent.BotMessage,140.727,null,2018-11-12T13:58:30.494+0000,null,null
2cpS4QCdLps7ycDeAxodOz,2cpS4QCdLps7ycDeAxodOz|0000054,MBFEvent.BotMessage,265.119,null,2018-11-12T13:58:34.337+0000,null,null
777Qsjs3i8qJyBr95zABbk,777Qsjs3i8qJyBr95zABbk|0000064,MBFEvent.BotMessage,754.275,null,2018-11-12T13:58:49.533+0000,null,null
2cpS4QCdLps7ycDeAxodOz,2cpS4QCdLps7ycDeAxodOz|0000059,MBFEvent.BotMessage,13.247,null,2018-11-12T13:58:49.960+0000,null,null
2cpS4QCdLps7ycDeAxodOz,2cpS4QCdLps7ycDeAxodOz|0000061,MBFEvent.BotMessage,582.275,null,2018-11-12T13:58:53.804+0000,null,null
777Qsjs3i8qJyBr95zABbk,777Qsjs3i8qJyBr95zABbk|0000071,MBFEvent.BotMessage,518.237,null,2018-11-12T13:58:58.597+0000,null,null
2cpS4QCdLps7ycDeAxodOz,2cpS4QCdLps7ycDeAxodOz|0000071,MBFEvent.BotMessage,756.257,null,2018-11-12T13:59:04.030+0000,null,null


#### Selection of records in df_4_final having eventName = 'MBFEvent.APICall'

In [34]:
df_api_responseTime = spark.sql(" \
          SELECT ConversationId as apiConversationId, ActivityId as apiActivityId, eventName as apiEventName, BotAnswerDuration as apiBotAnswerDuration, (apiTimeDiff) as apiApiTimeDiff, eventTime as apiEventTime, startTimeAPICall as apiStartTimeAPICall, endTimeAPICall as apiEndTimeAPICall  \
          FROM table_4_final \
          WHERE eventName = 'MBFEvent.APICall' \
          GROUP BY ConversationId, ActivityId, eventName, BotAnswerDuration, apiTimeDiff, eventTime, startTimeAPICall, endTimeAPICall ORDER BY eventTime  \
          ")
display(df_api_responseTime)

apiConversationId,apiActivityId,apiEventName,apiBotAnswerDuration,apiApiTimeDiff,apiEventTime,apiStartTimeAPICall,apiEndTimeAPICall
777Qsjs3i8qJyBr95zABbk,777Qsjs3i8qJyBr95zABbk|0000050,MBFEvent.APICall,0.0,203.145,2018-11-12T13:58:03.133+0000,2018-11-12T13:58:02.929+0000,2018-11-12T13:58:03.133+0000
777Qsjs3i8qJyBr95zABbk,777Qsjs3i8qJyBr95zABbk|0000050,MBFEvent.APICall,0.0,15.607,2018-11-12T13:58:03.148+0000,2018-11-12T13:58:03.133+0000,2018-11-12T13:58:03.148+0000
777Qsjs3i8qJyBr95zABbk,777Qsjs3i8qJyBr95zABbk|0000050,MBFEvent.APICall,0.0,15.602,2018-11-12T13:58:03.164+0000,2018-11-12T13:58:03.148+0000,2018-11-12T13:58:03.164+0000
777Qsjs3i8qJyBr95zABbk,777Qsjs3i8qJyBr95zABbk|0000050,MBFEvent.APICall,0.0,253.158,2018-11-12T13:58:03.417+0000,2018-11-12T13:58:03.164+0000,2018-11-12T13:58:03.417+0000
777Qsjs3i8qJyBr95zABbk,777Qsjs3i8qJyBr95zABbk|0000057,MBFEvent.APICall,0.0,209.7,2018-11-12T13:58:17.851+0000,2018-11-12T13:58:17.641+0000,2018-11-12T13:58:17.851+0000
777Qsjs3i8qJyBr95zABbk,777Qsjs3i8qJyBr95zABbk|0000057,MBFEvent.APICall,0.0,15.7,2018-11-12T13:58:17.867+0000,2018-11-12T13:58:17.851+0000,2018-11-12T13:58:17.867+0000
777Qsjs3i8qJyBr95zABbk,777Qsjs3i8qJyBr95zABbk|0000057,MBFEvent.APICall,0.0,15.61,2018-11-12T13:58:17.882+0000,2018-11-12T13:58:17.867+0000,2018-11-12T13:58:17.882+0000
777Qsjs3i8qJyBr95zABbk,777Qsjs3i8qJyBr95zABbk|0000057,MBFEvent.APICall,0.0,199.156,2018-11-12T13:58:18.081+0000,2018-11-12T13:58:17.882+0000,2018-11-12T13:58:18.081+0000
2cpS4QCdLps7ycDeAxodOz,2cpS4QCdLps7ycDeAxodOz|0000050,MBFEvent.APICall,0.0,62.533,2018-11-12T13:58:30.494+0000,2018-11-12T13:58:30.431+0000,2018-11-12T13:58:30.494+0000
2cpS4QCdLps7ycDeAxodOz,2cpS4QCdLps7ycDeAxodOz|0000050,MBFEvent.APICall,0.0,194.12,2018-11-12T13:58:32.914+0000,2018-11-12T13:58:32.720+0000,2018-11-12T13:58:32.914+0000


##### Join between Bot e api

In [36]:
df_join_responseTime = df_bot_responseTime.join(df_api_responseTime, ((df_bot_responseTime.botConversationId == df_api_responseTime.apiConversationId) & ((df_bot_responseTime.botActivityId == df_api_responseTime.apiActivityId))), 'left')
#df_bot_responseTime.unpersist() # unpersist() useful to prevent duplicates
#df_api_responseTime.unpersist()
df_join_responseTime_final = df_join_responseTime.where((col("botEventTime") >= col("apiEndTimeAPICall")) ).orderBy('apiEventTime') # Checking that the call of API  ends before the BotAnswer for the same ActivityId
#df_join_responseTime.unpersist()
df_join_responseTime_final= df_join_responseTime_final.select('botConversationId',
 'botActivityId',
 'botEventName',
 'botBotAnswerDuration',
 'apiApiTimeDiff',
 'botEventTime',
 'apiEventTime',
 'apiStartTimeAPICall',
 'apiEndTimeAPICall')
#display(df_join_responseTime_final.select('botConversationId','botActivityId','botEventTime','apiStartTimeAPICall','apiEndTimeAPICall'))

display(df_join_responseTime_final)

""" Replacing NULLS with zeros"""  # <-- LASCIARLO SOLO SE VOGLIO CAMBIARE I NULL DI APITIMEDIFF CON DEGLI ZERO!!!!
"""udf = UserDefinedFunction(lambda x: float(0) if ClassType(x) != float else x, FloatType() )
df_join_responseTime_final= df_join_responseTime_final.select(*[udf(column).alias('apiApiTimeDiff') if column == 'apiApiTimeDiff' else column for column in df_join_responseTime_final.columns])"""

botConversationId,botActivityId,botEventName,botBotAnswerDuration,apiApiTimeDiff,botEventTime,apiEventTime,apiStartTimeAPICall,apiEndTimeAPICall
2cpS4QCdLps7ycDeAxodOz,2cpS4QCdLps7ycDeAxodOz|0000050,MBFEvent.BotMessage,140.727,62.533,2018-11-12T13:58:30.494+0000,2018-11-12T13:58:30.494+0000,2018-11-12T13:58:30.431+0000,2018-11-12T13:58:30.494+0000
2cpS4QCdLps7ycDeAxodOz,2cpS4QCdLps7ycDeAxodOz|0000054,MBFEvent.BotMessage,265.119,218.752,2018-11-12T13:58:34.337+0000,2018-11-12T13:58:34.321+0000,2018-11-12T13:58:34.102+0000,2018-11-12T13:58:34.321+0000
2cpS4QCdLps7ycDeAxodOz,2cpS4QCdLps7ycDeAxodOz|0000075,MBFEvent.BotMessage,109.42,46.866,2018-11-12T13:59:07.023+0000,2018-11-12T13:59:07.023+0000,2018-11-12T13:59:06.976+0000,2018-11-12T13:59:07.023+0000
2cpS4QCdLps7ycDeAxodOz,2cpS4QCdLps7ycDeAxodOz|0000079,MBFEvent.BotMessage,252.59,190.1,2018-11-12T13:59:10.845+0000,2018-11-12T13:59:10.814+0000,2018-11-12T13:59:10.624+0000,2018-11-12T13:59:10.814+0000
2cpS4QCdLps7ycDeAxodOz,2cpS4QCdLps7ycDeAxodOz|0000088,MBFEvent.BotMessage,138.718,30.28,2018-11-12T14:04:05.617+0000,2018-11-12T14:04:05.617+0000,2018-11-12T14:04:05.587+0000,2018-11-12T14:04:05.617+0000
2cpS4QCdLps7ycDeAxodOz,2cpS4QCdLps7ycDeAxodOz|0000092,MBFEvent.BotMessage,485.423,420.925,2018-11-12T14:04:11.087+0000,2018-11-12T14:04:11.039+0000,2018-11-12T14:04:10.618+0000,2018-11-12T14:04:11.039+0000
2cpS4QCdLps7ycDeAxodOz,2cpS4QCdLps7ycDeAxodOz|0000100,MBFEvent.BotMessage,249.043,175.186,2018-11-12T14:04:16.292+0000,2018-11-12T14:04:16.292+0000,2018-11-12T14:04:16.117+0000,2018-11-12T14:04:16.292+0000
2cpS4QCdLps7ycDeAxodOz,2cpS4QCdLps7ycDeAxodOz|0000104,MBFEvent.BotMessage,252.643,205.699,2018-11-12T14:04:20.438+0000,2018-11-12T14:04:20.422+0000,2018-11-12T14:04:20.217+0000,2018-11-12T14:04:20.422+0000
JEBXXIfay3m328mWQLvL0x,JEBXXIfay3m328mWQLvL0x|0000018,MBFEvent.BotMessage,310.325,202.42,2018-11-12T14:08:07.494+0000,2018-11-12T14:08:07.494+0000,2018-11-12T14:08:07.292+0000,2018-11-12T14:08:07.494+0000
JEBXXIfay3m328mWQLvL0x,JEBXXIfay3m328mWQLvL0x|0000022,MBFEvent.BotMessage,241.582,192.933,2018-11-12T14:08:11.297+0000,2018-11-12T14:08:11.266+0000,2018-11-12T14:08:11.073+0000,2018-11-12T14:08:11.266+0000


##### Sum apiTimeDiff for the same activity: subtracting sum from BotAnswerDuration

In [38]:
df_join_responseTime_final.createOrReplaceTempView('table_join_final')

df_sum_api= spark.sql( "SELECT botConversationId, botActivityId, botEventName, botBotAnswerDuration,botEventTime, sum(apiApiTimeDiff) as sumApiTimeDiff\
            FROM table_join_final \
            GROUP BY botConversationId, botActivityId, botEventName, botBotAnswerDuration,botEventTime")


df_sum_api_final = df_sum_api.withColumn("botBotAnswerDuration", (df_sum_api["botBotAnswerDuration"] - df_sum_api["sumApiTimeDiff"]))
df_sum_api_final= df_sum_api_final.drop("sumApiTimeDiff")
display(df_sum_api)

botConversationId,botActivityId,botEventName,botBotAnswerDuration,botEventTime,sumApiTimeDiff
JEBXXIfay3m328mWQLvL0x,JEBXXIfay3m328mWQLvL0x|0000160,MBFEvent.BotMessage,124.324,2018-11-12T14:41:04.457+0000,15.60200023651123
JEBXXIfay3m328mWQLvL0x,JEBXXIfay3m328mWQLvL0x|0000018,MBFEvent.BotMessage,310.325,2018-11-12T14:08:07.494+0000,202.4199981689453
JEBXXIfay3m328mWQLvL0x,JEBXXIfay3m328mWQLvL0x|0000229,MBFEvent.BotMessage,422.194,2018-11-12T14:51:17.789+0000,218.70199584960938
8mkbRBVLXnqB1DyyZ1pVcZ,8mkbRBVLXnqB1DyyZ1pVcZ|0000551,MBFEvent.BotMessage,593.452,2018-11-12T14:11:20.744+0000,378.0299987792969
JEBXXIfay3m328mWQLvL0x,JEBXXIfay3m328mWQLvL0x|0000250,MBFEvent.BotMessage,792.632,2018-11-12T14:54:29.298+0000,358.2330017089844
JEBXXIfay3m328mWQLvL0x,JEBXXIfay3m328mWQLvL0x|0000043,MBFEvent.BotMessage,2826.354,2018-11-12T14:09:18.227+0000,855.3159790039062
2cpS4QCdLps7ycDeAxodOz,2cpS4QCdLps7ycDeAxodOz|0000050,MBFEvent.BotMessage,140.727,2018-11-12T13:58:30.494+0000,62.53300094604492
8mkbRBVLXnqB1DyyZ1pVcZ,8mkbRBVLXnqB1DyyZ1pVcZ|0000712,MBFEvent.BotMessage,356.439,2018-11-12T14:55:47.439+0000,217.91200256347656
JEBXXIfay3m328mWQLvL0x,JEBXXIfay3m328mWQLvL0x|0000047,MBFEvent.BotMessage,660.223,2018-11-12T14:09:28.110+0000,221.41400146484375
2cpS4QCdLps7ycDeAxodOz,2cpS4QCdLps7ycDeAxodOz|0000100,MBFEvent.BotMessage,249.043,2018-11-12T14:04:16.292+0000,175.18600463867188


In [39]:
display(df_sum_api_final)

botConversationId,botActivityId,botEventName,botBotAnswerDuration,botEventTime
JEBXXIfay3m328mWQLvL0x,JEBXXIfay3m328mWQLvL0x|0000160,MBFEvent.BotMessage,108.72199726104736,2018-11-12T14:41:04.457+0000
JEBXXIfay3m328mWQLvL0x,JEBXXIfay3m328mWQLvL0x|0000018,MBFEvent.BotMessage,107.90501403808594,2018-11-12T14:08:07.494+0000
JEBXXIfay3m328mWQLvL0x,JEBXXIfay3m328mWQLvL0x|0000229,MBFEvent.BotMessage,203.49200439453125,2018-11-12T14:51:17.789+0000
8mkbRBVLXnqB1DyyZ1pVcZ,8mkbRBVLXnqB1DyyZ1pVcZ|0000551,MBFEvent.BotMessage,215.42202758789062,2018-11-12T14:11:20.744+0000
JEBXXIfay3m328mWQLvL0x,JEBXXIfay3m328mWQLvL0x|0000250,MBFEvent.BotMessage,434.3990173339844,2018-11-12T14:54:29.298+0000
JEBXXIfay3m328mWQLvL0x,JEBXXIfay3m328mWQLvL0x|0000043,MBFEvent.BotMessage,1971.0380249023438,2018-11-12T14:09:18.227+0000
2cpS4QCdLps7ycDeAxodOz,2cpS4QCdLps7ycDeAxodOz|0000050,MBFEvent.BotMessage,78.19400405883789,2018-11-12T13:58:30.494+0000
8mkbRBVLXnqB1DyyZ1pVcZ,8mkbRBVLXnqB1DyyZ1pVcZ|0000712,MBFEvent.BotMessage,138.52699279785156,2018-11-12T14:55:47.439+0000
JEBXXIfay3m328mWQLvL0x,JEBXXIfay3m328mWQLvL0x|0000047,MBFEvent.BotMessage,438.80902099609375,2018-11-12T14:09:28.110+0000
2cpS4QCdLps7ycDeAxodOz,2cpS4QCdLps7ycDeAxodOz|0000100,MBFEvent.BotMessage,73.85699462890625,2018-11-12T14:04:16.292+0000


##### Updating df_5_final with the rectified BotAnswerDuration (same schema)

In [ ]:
df_5_final = df_4_final.join(df_sum_api_final, (df_4_final.conversationId == df_sum_api_final.botConversationId) & (df_4_final.activityId == df_sum_api_final.botActivityId) & (df_4_final.eventName == df_sum_api_final.botEventName) & (df_4_final.eventTime == df_sum_api_final.botEventTime), 'left').select('itemId',
 'ClientAtHome',
 'LUISentities',
 'LUISintent',
 'LUISscore',
 'ProblemSolved',
 'QNAanswer',
 'QNAcategory',
 'QNAscore',
 'QNAsource',
 'QNAuserMessage',
 'TXTANALYTICSscore',
 'activityId',
 'channel',
 'conversationId',
 'currentScenario',
 'diagnosticResponse',
 'dialogCloseType',
 'endTimeAPICall',
 'eventName',
 'eventTime',
 'eventdate',
 'isAuthenticated',
 'isLeaf',
 'mainScenario',
 'modemAlignment',
 'modemChanged',
 'modemType',
 'notUnderstoodMessage',
 'reason',
 'scenario',
 'scenarioCalled',
 'scenarioId',
 'startTimeAPICall',
 'telemetryId',
 'text',
 'ticket',
 'ticketForm',
 'timestamp',
 'toolExec',
 'toolResult',
 'user',
 'userId',
 'userName',
 'applicationInsight',
 'businessUnit',
 'type',
 'apiTimeDiff',
 'BotAnswerDuration',
 'botBotAnswerDuration',
 'EventTimePrevious',
 'DateUTCInsertRow').orderBy('eventTime')

#df_4_final.unpersist()   #avoiding duplicates
# relevant columns to display: .select('conversationId', 'activityId', 'botBotAnswerDuration', 'BotAnswerDuration', 'apiTimeDiff', 'eventTime' ).orderBy('eventTime'))

#there is the original BotAnswerDuration to make comparisons

#display(df_5_final)

In [ ]:
#display(df_5_final.select(['botBotAnswerDuration', 'BotAnswerDuration' ]))   #comparison between original and new BotAnswerDuration
df_5_final= df_5_final.withColumn('botBotAnswerDuration', df_5_final['botBotAnswerDuration'].cast(FloatType()))
df_5_final= df_5_final.withColumn('BotAnswerDuration', df_5_final['BotAnswerDuration'].cast(FloatType()))
#df_5_final.printSchema()

def bot_duration(old,new):
    if new is None and old>0:
        return old
    else:
        return new
  

udf_bot_duration = UserDefinedFunction(bot_duration, FloatType())
df_5_final= df_5_final.withColumn('BotAnswerDuration', udf_bot_duration(df_5_final['BotAnswerDuration'], df_5_final['botBotAnswerDuration']))
#display(df_5_final)

##### With this cell it is over the retification of BotAnswerDuration

In [44]:
df_5_final= df_5_final.drop('botBotAnswerDuration')
df_5_final.schema== df_4_final.schema   #check if the schema is the same of the original (the one before rectification)

Out[ 161 ]: False

In [45]:
#df_5_final.persist()

In [46]:
#06-12-2018 to be added Environment and eventNameDecode correctly
"""df_5_final = df_5_final.withColumn('Environment', F.lit(environment))
df_5_final = df_5_final.withColumn('eventNameDecode', when(df_5_final['eventName'].like('%MBFEvent%') , df_5_final['eventName'].substr(10,15)).otherwise(df_5_final['eventName']))"""

Out[ 163 ]: "df_5_final = df_5_final.withColumn('Environment', F.lit(environment))\ndf_5_final = df_5_final.withColumn('eventNameDecode', when(df_5_final['eventName'].like('%MBFEvent%') , df_5_final['eventName'].substr(10,15)).otherwise(df_5_final['eventName']))"

added Environment and eventNameDecode

In [48]:
df_5_final = df_5_final.withColumn('Environment', F.lit(environment))
df_5_final = df_5_final.withColumn('eventNameDecode', when(df_5_final['eventName'].like('%MBFEvent%') , df_5_final['eventName'].substr(10,15)).otherwise(df_5_final['eventName']))

In [49]:
df_5_final.count()

Out[ 165 ]: 921

### Adding new fields to Events: Status, Recognition, Status Code, Recognition Code

In [ ]:
#new_Events = df_5_final.filter(df_5_final.type == 'new')
new_Events= df_5_final
new_Events= new_Events.withColumn('Status', when( ((new_Events.mainScenario != 'Default Dialog') & (new_Events.scenario== 'InitScenario')) | (
  (new_Events.eventName== 'MBFEvent.QNAEvent') & (new_Events.QNAcategory!= 'ChitChat')), F.lit('in corso')).when(new_Events.mainScenario == 'Default Dialog', F.lit('in corso')).when((new_Events.scenario== 'InitScenario') | ((new_Events.eventName== 'MBFEvent.QNAEvent') & (new_Events.QNAcategory== 'ChitChat')), F.lit('in corso')).otherwise(F.lit('undefined')))

new_Events= new_Events.withColumn('Recognition', when( ((new_Events.mainScenario != 'Default Dialog') & (new_Events.scenario== 'InitScenario')) | (
  (new_Events.eventName== 'MBFEvent.QNAEvent') & (new_Events.QNAcategory!= 'ChitChat')), F.lit('riconosciuta')).when(new_Events.mainScenario == 'Default Dialog', F.lit('non riconosciuta')).when((new_Events.scenario== 'InitScenario') | ((new_Events.eventName== 'MBFEvent.QNAEvent') & (new_Events.QNAcategory== 'ChitChat')), F.lit('inconsistente')).otherwise(F.lit('undefined')))

new_Events= new_Events.withColumn('Status code', when( ((new_Events.mainScenario != 'Default Dialog') & (new_Events.scenario== 'InitScenario')) | (
  (new_Events.eventName== 'MBFEvent.QNAEvent') & (new_Events.QNAcategory!= 'ChitChat')), F.lit('IC')).when(new_Events.mainScenario == 'Default Dialog', F.lit('IC')).when((new_Events.scenario== 'InitScenario') | ((new_Events.eventName== 'MBFEvent.QNAEvent') & (new_Events.QNAcategory== 'ChitChat')), F.lit('IC')).otherwise(F.lit('undefined')))

new_Events= new_Events.withColumn('Recognition code', when( ((new_Events.mainScenario != 'Default Dialog') & (new_Events.scenario== 'InitScenario')) | (
  (new_Events.eventName== 'MBFEvent.QNAEvent') & (new_Events.QNAcategory!= 'ChitChat')), F.lit('RIC')).when(new_Events.mainScenario == 'Default Dialog', F.lit('NRIC')).when((new_Events.scenario== 'InitScenario') | ((new_Events.eventName== 'MBFEvent.QNAEvent') & (new_Events.QNAcategory== 'ChitChat')), F.lit('INC')).otherwise(F.lit('undefined')))

display(new_Events)
#new_Events.count()
#print(new_Events.explain())

##### Adding to Events table a temporary field which says the difference (in hours) between the actual time and the EventTime

In [ ]:
def timestamp_diff_hours(endTime,startTime):
    if startTime is None or endTime is None:
        return 0
    tt = timestamp_to_long(endTime) - timestamp_to_long(startTime)
    return tt / (1000)
  
def arrotonda(x):
    return round(x,2)
  
func_udf2= udf(timestamp_diff_hours, FloatType())
new_Events= new_Events.withColumn('diff_hours', func_udf2(current_timestamp(), new_Events['eventTime'])/(1000*3600))

udf_round= udf(arrotonda, FloatType())
new_Events= new_Events.withColumn('diff_hours', udf_round(new_Events['diff_hours']))

#display(new_Events)

#### Filtering Events in standby (between 20 and 30 minutes) and after standby (more than 30 minutes)

In [55]:
new_Events.createOrReplaceTempView('new_Events_sql')

stand_by = spark.sql(" \
          SELECT ConversationId as convId, max(eventTime) as last_time_conv \
          FROM new_Events_sql \
          WHERE diff_hours*60 >= 20 AND diff_hours*60 <30 \
          GROUP BY ConversationId \
          ORDER BY max(eventTime)\
          ")

#ATTENZIONE: L'UPPER BOUND DELLO STANBY DEVE ESSERE IL LOWER BOUND DI "OLTRE_STAN_BY, E UNO CHE LO INCLUDE E L ALTRO NO, PER COPRIRE TUTTI I CASI"
after_stand_by = spark.sql(" \
          SELECT ConversationId as convId, max(eventTime) as last_time_conv \
          FROM new_Events_sql \
          WHERE diff_hours*60 >= 30 \
          GROUP BY ConversationId \
          ORDER BY max(eventTime)\
          ")

stand_by= stand_by.withColumn('Status_temp', F.lit('stand by'))
after_stand_by= after_stand_by.withColumn('Status_temp', F.lit('oltre stand by'))
union_sb_and_after= stand_by.unionAll(after_stand_by)
display(union_sb_and_after)  #avrò l ultimo record di ogni conv che è oltre lo standby, ovvero oltre i 30 min

convId,last_time_conv,Status_temp
777Qsjs3i8qJyBr95zABbk,2018-11-12T13:59:32.347+0000,oltre stand by
2cpS4QCdLps7ycDeAxodOz,2018-11-12T14:07:45.439+0000,oltre stand by
JEBXXIfay3m328mWQLvL0x,2018-11-12T14:55:34.613+0000,oltre stand by
8mkbRBVLXnqB1DyyZ1pVcZ,2018-11-12T14:57:54.130+0000,oltre stand by


#### Joining table Events with the one filtered in the previous cell, referring to all conversations in SB and after

In [ ]:
new_Events2= new_Events.join(union_sb_and_after, ((new_Events.conversationId == union_sb_and_after.convId) & (new_Events.eventTime == union_sb_and_after.last_time_conv)
                        ), 'left')

new_Events2= new_Events2.withColumn('Status', when(((new_Events2['Status_temp']== 'oltre stand by') & (new_Events2['isLeaf'] == 'true')), F.lit('chiusa')).when(((new_Events2['Status_temp']== 'oltre stand by') & (new_Events2['isLeaf'] !='true')), F.lit('abbandonata')).otherwise(new_Events2['Status'])).withColumn('Status code', when(((new_Events2['Status_temp']== 'oltre stand by') & (new_Events2['isLeaf'] == 'true')), F.lit('CH')).when(((new_Events2['Status_temp']== 'oltre stand by') & (new_Events2['isLeaf'] !='true')), F.lit('AB')).otherwise(new_Events2['Status code']))

new_Events2= new_Events2.withColumnRenamed("Status code", "StatusCode").withColumnRenamed("Recognition code", "RecognitionCode")

#display(new_Events2.select('isLeaf','Status_temp','Status','StatusCode', 'Recognition', col('RecognitionCode').alias('ProvaRename')))
#display(new_Events2)

In [58]:
new_Events2.count()

Out[ 170 ]: 921

In [59]:
#display(new_Events2.where(col('conversationId') == 'BEDTFSDcWqa6rfGc9PwiKP').orderBy('eventTime'))   
display(new_Events2.filter((new_Events2.Status=='abbandonata') | (new_Events2.Status=='chiusa'))) 
# equivalente: display(new_Events2.filter(new_Events2.Status_temp=='oltre stand by')), perchè chiusa o abbandonata sono oltre sb

itemId,ClientAtHome,LUISentities,LUISintent,LUISscore,ProblemSolved,QNAanswer,QNAcategory,QNAscore,QNAsource,QNAuserMessage,TXTANALYTICSscore,activityId,channel,conversationId,currentScenario,diagnosticResponse,dialogCloseType,endTimeAPICall,eventName,eventTime,eventdate,isAuthenticated,isLeaf,mainScenario,modemAlignment,modemChanged,modemType,notUnderstoodMessage,reason,scenario,scenarioCalled,scenarioId,startTimeAPICall,telemetryId,text,ticket,ticketForm,timestamp,toolExec,toolResult,user,userId,userName,applicationInsight,businessUnit,type,apiTimeDiff,BotAnswerDuration,EventTimePrevious,DateUTCInsertRow,Environment,eventNameDecode,Status,Recognition,StatusCode,RecognitionCode,diff_hours,convId,last_time_conv,Status_temp
5acf0b77-e684-11e8-8c74-1df3eaf4721b,,,,,,,,,,,null,2cpS4QCdLps7ycDeAxodOz|0000116,pat,2cpS4QCdLps7ycDeAxodOz,WO_Problemi_Navigazione,,,null,MBFEvent.BotMessage,2018-11-12T14:07:45.439+0000,2018-11-12,false,false,WO_Inizio,,,,,,,,adf415c5-3843-4587-9e04-e28ebef74e63,null,,Qual è il tuo problema?,,,2018-11-12T14:07:45.439+0000,,,,dev-timva-wo-botservice@-164JqvoUec,dev-timva-wo-botservice,appinsights-wo,wo,new,null,855.895,2018-11-12T14:07:44.583+0000,2018-12-19T17:26:16.348+0000,DEV,BotMessage,abbandonata,undefined,AB,undefined,890.31,2cpS4QCdLps7ycDeAxodOz,2018-11-12T14:07:45.439+0000,oltre stand by
0d572e21-e68b-11e8-a351-0f405d5c52f9,,,,,,,,,,,null,0b7dab48-7c1e-415b-8093-6b11b98095bc,pat,JEBXXIfay3m328mWQLvL0x,Feedback Dialog,,,null,MBFEvent.BotMessage,2018-11-12T14:55:34.613+0000,2018-11-12,false,false,WO_Inizio,,,,,,,,54188222-5a83-424b-baed-1e1438772a90,null,,Sono stato utile?,,,2018-11-12T14:55:34.595+0000,,,,dev-timva-wo-botservice@-164JqvoUec,dev-timva-wo-botservice,appinsights-wo,wo,new,null,null,null,2018-12-19T17:26:16.348+0000,DEV,BotMessage,abbandonata,undefined,AB,undefined,889.51,JEBXXIfay3m328mWQLvL0x,2018-11-12T14:55:34.613+0000,oltre stand by
55cd7162-e68b-11e8-bf7a-5fba8834ae28,,,,,,,,,,,null,8mkbRBVLXnqB1DyyZ1pVcZ|0000723,pat,8mkbRBVLXnqB1DyyZ1pVcZ,Welcome,,,null,MBFEvent.BotMessage,2018-11-12T14:57:54.130+0000,2018-11-12,true,true,Reset Dialog,,,,,,,,cea8d6a0-d975-4129-b098-9892bb739e29,null,,"Ciao, sono l'assistente virtuale dell'assistenza tecnica. Come posso aiutarti?",,,2018-11-12T14:57:54.121+0000,,,FE054A8741794EEA06252DD6DB61F8DA,dev-timva-wo-botservice@-164JqvoUec,dev-timva-wo-botservice,appinsights-wo,wo,new,null,null,null,2018-12-19T17:26:16.348+0000,DEV,BotMessage,chiusa,undefined,CH,undefined,889.47,8mkbRBVLXnqB1DyyZ1pVcZ,2018-11-12T14:57:54.130+0000,oltre stand by
3b79ea7b-e683-11e8-8c74-1df3eaf4721b,,,,,,,,,,,null,777Qsjs3i8qJyBr95zABbk|0000097,pat,777Qsjs3i8qJyBr95zABbk,,,,2018-11-12T13:59:32.347+0000,MBFEvent.APICall,2018-11-12T13:59:32.347+0000,2018-11-12,,,WO_Reset_Porta_Voip,,,,,,,,6903fb36-c709-4669-82d0-1a69c9c65fae,2018-11-12T13:59:32.139+0000,,,,,2018-11-12T13:59:28.942+0000,,,,5f9d9ab4e3fcaecdabbed1228ffaacab7f942b91,,appinsights-wo,wo,new,207.536,null,null,2018-12-19T17:26:16.348+0000,DEV,APICall,abbandonata,undefined,AB,undefined,890.45,777Qsjs3i8qJyBr95zABbk,2018-11-12T13:59:32.347+0000,oltre stand by
3b79ea7c-e683-11e8-8c74-1df3eaf4721b,,,,,,,,,,,null,777Qsjs3i8qJyBr95zABbk|0000097,pat,777Qsjs3i8qJyBr95zABbk,WO_Reset_Porta_Voip,,,null,MBFEvent.BotMessage,2018-11-12T13:59:32.347+0000,2018-11-12,false,false,WO_Reset_Porta_Voip,,,,,,,,6903fb36-c709-4669-82d0-1a69c9c65fae,null,,"Stiamo effettuando un reset in centrale. Spegni e riaccendi il modem, attendi che le spie siano di nuovo accese. Dimmi quando hai finito.",,,2018-11-12T13:59:32.347+0000,,,,dev-timva-wo-botservice@-164JqvoUec,dev-timva-wo-botservice,appinsights-wo,wo,new,null,null,null,2018-12-19T17:26:16.348+0000,DEV,BotMessage,abbandonata,undefined,AB,undefined,890.45,777Qsjs3i8qJyBr95zABbk,2018-11-12T13:59:32.347+0000,oltre stand by


In [60]:
new_Events2= new_Events2.drop('convId', 'last_time_conv', 'Status_temp')
new_Events2= new_Events2.withColumnRenamed("diff_hours", "HourFromLastEvent")

In [61]:
new_Events2= new_Events2.withColumn('EventHourMinute' , new_Events2['eventTime'].cast(StringType()))
#display(new_Events2)

In [ ]:
def hour_minute(x):
    return  ''.join(x.split()[1][:5].split(':'))                         

udf_hour_minute= UserDefinedFunction(hour_minute, StringType())
new_Events2= new_Events2.withColumn('EventHourMinute' , udf_hour_minute(new_Events2['EventHourMinute']))
#display(new_Events2)
#new_Events2.persist() #riabilitato alle 19:38 del 30/11/2018 dopo il Run Manuale del 2018-11-30/10 partito alle 19:30 (osserviamo se cambia qualcosa con il successivo RUN)

In [63]:
"""PROCEDIMENTO: 
INPUT: Lettura degli eventi correnti (slot orario corrente)
- giro classico per "event" (quello che già facciamo adesso)
- NUOVO giro per "conversation":
              1.1- lettura della delta table "conversation" (in cui avremo solo type = "previous") e mettiamo tutto in un Dataframe
               1.2- FULL JOIN del Dataframe (quindi dei record della delta table "conversation", solo type = "previous") tra il DF letto e il DF degli eventi correnti, ON conversation.conversationId = event.conversationId (eventi correnti)
               1.3 - determinare il type (F.lit.type) grazie alla FULL JOIN di cui sopra e scrivo in un DF
               1.4 - partendo dal DF del punto 1.3, vado in JOIN con gli eventi correnti per fare le aggregazioni al fine di aggiornare i vari campi della conversazione come ad esempio lo "stato" della conversazione, il "contatore dei messaggi totali", dei "BotMessage", "DataChiusura" etc...
                              N.B. lo "stato" della conversazione sarà ridefinito con la lettura del MAX degli eventi, invece un eventuale contatore (ad esempio TotMessaggi) sarà la Somma del campo letto dalla delta table e la somma del campo determinato dall'aggregazione degli eventi correnti               
N.B. a questo punto avremo conversazioni in Stato "COLD" e "HOT"
               1.5 - inviare al DW quelle che sono diventate COLD ("chiuse" e "abbandonate")
               1.6 - scrittura in Delta Table "Conversation" delle conversazioni rimanenti, ossia le "HOT ("In corso" e "Stand by")
"""

Out[ 175 ]: 'PROCEDIMENTO: \nINPUT: Lettura degli eventi correnti (slot orario corrente)\n- giro classico per "event" (quello che già facciamo adesso)\n- NUOVO giro per "conversation":\n 1.1- lettura della delta table "conversation" (in cui avremo solo type = "previous") e mettiamo tutto in un Dataframe\n 1.2- FULL JOIN del Dataframe (quindi dei record della delta table "conversation", solo type = "previous") tra il DF letto e il DF degli eventi correnti, ON conversation.conversationId = event.conversationId (eventi correnti)\n 1.3 - determinare il type (F.lit.type) grazie alla FULL JOIN di cui sopra e scrivo in un DF\n 1.4 - partendo dal DF del punto 1.3, vado in JOIN con gli eventi correnti per fare le aggregazioni al fine di aggiornare i vari campi della conversazione come ad esempio lo "stato" della conversazione, il "contatore dei messaggi totali", dei "BotMessage", "DataChiusura" etc...\n N.B. lo "stato" della conversazione sarà ridefinito con la lettura del MAX degli eventi, invece un eventuale contatore (ad esempio TotMessaggi) sarà la Somma del campo letto dalla delta table e la somma del campo determinato dall\'aggregazione degli eventi correnti \nN.B. a questo punto avremo conversazioni in Stato "COLD" e "HOT"\n 1.5 - inviare al DW quelle che sono diventate COLD ("chiuse" e "abbandonate")\n 1.6 - scrittura in Delta Table "Conversation" delle conversazioni rimanenti, ossia le "HOT ("In corso" e "Stand by")\n'

###Taking last event of each Conversation. Rather than ItemId it is considered a Key the couple [ConvId, max(eventime)]

In [65]:
df_1.select('ConversationId').distinct().count()   # 6LRLPjJDLkQ85QplizAeFS, PIPPO e ALDO

In [66]:
new_Events2.select('ConversationId').distinct().count()  #una sola: 6LRLPjJDLkQ85QplizAeFS

In [67]:
display(new_Events2.select('ConversationId').groupBy("ConversationId").count())

In [68]:
new_Events2.createOrReplaceTempView("Events_final")
# è l'ultimo evento di ogni conversazione

last_record = spark.sql(" \
          SELECT ConversationId as convId, max(eventTime) as last_time_conv, min(eventTime) as ConversationStartTime \
          FROM Events_final \
          GROUP BY ConversationId \
          ORDER BY max(eventTime)\
          ")

display(last_record)

convId,last_time_conv,ConversationStartTime
777Qsjs3i8qJyBr95zABbk,2018-11-12T13:59:32.347+0000,2018-11-12T13:58:00.081+0000
2cpS4QCdLps7ycDeAxodOz,2018-11-12T14:07:45.439+0000,2018-11-12T13:57:54.672+0000
JEBXXIfay3m328mWQLvL0x,2018-11-12T14:55:34.613+0000,2018-11-12T14:07:54.379+0000
8mkbRBVLXnqB1DyyZ1pVcZ,2018-11-12T14:57:54.130+0000,2018-11-12T14:09:18.462+0000


#### Create Conversations Table: Joining previous cells with Events to take the other fields I need

In [70]:
#cold_conversations= conv_after_stand_by.join(new_Events2, ((new_Events2.conversationId == conv_after_stand_by.convId) & (new_Events2.eventTime == conv_after_stand_by.last_time_conv)
                        #), 'left')

#oppure
conversations = new_Events2.join(last_record, ((new_Events2.conversationId == last_record.convId) & (new_Events2.eventTime == last_record.last_time_conv)
                        ), 'right').select("ConversationId", 'StatusCode', 'Status','last_time_conv', 'RecognitionCode', 'Recognition', 'ConversationStartTime')

conversations= conversations.withColumn('conversationStartDate' , conversations['ConversationStartTime'].cast(StringType()) )

"""def data(x):
  return x[:10]

udf_data= UserDefinedFunction(data, StringType())
conversations= conversations.withColumn('conversationStartDate', udf_data(conversations['conversationStartDate']))"""

#Alternativa senza udf
conversations= conversations.withColumn('conversationStartDate', conversations['conversationStartDate'].cast(StringType()).substr(1,10))

conversations= conversations.select(col("ConversationId"), "ConversationStartDate", 'ConversationStartTime', col('last_time_conv').alias('ConversationEndTime'), 
                                    col('StatusCode').alias('ConversationStatusCode'), col('Status').alias('ConversationStatus'), 
                         col('RecognitionCode').alias('ConversationRecognitionCode'), col('Recognition').alias('ConversationRecognition'))


display(conversations)

ConversationId,ConversationStartDate,ConversationStartTime,ConversationEndTime,ConversationStatusCode,ConversationStatus,ConversationRecognitionCode,ConversationRecognition
2cpS4QCdLps7ycDeAxodOz,2018-11-12,2018-11-12T13:57:54.672+0000,2018-11-12T14:07:45.439+0000,AB,abbandonata,undefined,undefined
JEBXXIfay3m328mWQLvL0x,2018-11-12,2018-11-12T14:07:54.379+0000,2018-11-12T14:55:34.613+0000,AB,abbandonata,undefined,undefined
8mkbRBVLXnqB1DyyZ1pVcZ,2018-11-12,2018-11-12T14:09:18.462+0000,2018-11-12T14:57:54.130+0000,CH,chiusa,undefined,undefined
777Qsjs3i8qJyBr95zABbk,2018-11-12,2018-11-12T13:58:00.081+0000,2018-11-12T13:59:32.347+0000,AB,abbandonata,undefined,undefined
777Qsjs3i8qJyBr95zABbk,2018-11-12,2018-11-12T13:58:00.081+0000,2018-11-12T13:59:32.347+0000,AB,abbandonata,undefined,undefined


In [71]:
conversations.printSchema()

root
-- ConversationId: string (nullable = true)
-- ConversationStartDate: string (nullable = true)
-- ConversationStartTime: timestamp (nullable = true)
-- ConversationEndTime: timestamp (nullable = true)
-- ConversationStatusCode: string (nullable = true)
-- ConversationStatus: string (nullable = true)
-- ConversationRecognitionCode: string (nullable = true)
-- ConversationRecognition: string (nullable = true)

In [ ]:
"""for i in range(1, 18):
    if i<=0:
      pass
    if i>=1 and i<=9:
      try:
        print('There are {} files in this day'.format(len(dbutils.fs.ls("/delta/"+'QA'+"/Events/eventDate=2018-12-0" +str(i) ))))
      except:
        print('there are no files for day ', i)
    elif i>=10:
      try:
        print('There are {} files in this day'.format(len(dbutils.fs.ls("/delta/"+'QA'+"/Events/eventDate=2018-12-" +str(i) ))))
      except:
        print('there are no files for day ', i)"""

#### Conversations of last hour (only the ID of the distinct ones)

In [74]:
#display(new_Events2.select('ConversationId').groupBy("ConversationId").count())
lastHourConversations= conversations.select(col('conversationId').alias('NewConvId'), col('ConversationStartDate').alias('NewStartDate'), col('ConversationStartTime').alias('NewStartTime'),
                             col('ConversationEndTime').alias('NewEndTime') ).distinct()
display(lastHourConversations)

NewConvId,NewStartDate,NewStartTime,NewEndTime
2cpS4QCdLps7ycDeAxodOz,2018-11-12,2018-11-12T13:57:54.672+0000,2018-11-12T14:07:45.439+0000
JEBXXIfay3m328mWQLvL0x,2018-11-12,2018-11-12T14:07:54.379+0000,2018-11-12T14:55:34.613+0000
8mkbRBVLXnqB1DyyZ1pVcZ,2018-11-12,2018-11-12T14:09:18.462+0000,2018-11-12T14:57:54.130+0000
777Qsjs3i8qJyBr95zABbk,2018-11-12,2018-11-12T13:58:00.081+0000,2018-11-12T13:59:32.347+0000


### Import previous conversations from delta. Union previous and new

In [76]:
previous_conversations = spark.read.format("delta").load("/delta/"+environment+"/Conversations/")
previous_conversations = previous_conversations.withColumn("recordType", F.lit("previous conv"))
"""# Union of two DataFrames
#unionDataFrame = previous_events.unionAll(df_1)"""
display(previous_conversations)   

ConversationId,ConversationStartDate,ConversationStartTime,ConversationEndTime,ConversationStatusCode,ConversationStatus,ConversationRecognitionCode,ConversationRecognition,recordType
1CY3JeMvsiCGxl3HQfFRXA,2018-12-19,2018-12-19T10:04:17.036+0000,2018-12-19T10:04:19.767+0000,AB,abbandonata,undefined,undefined,previous conv
5RCYVLGONapLew3SD2WrBD,2018-12-19,2018-12-19T10:04:37.923+0000,2018-12-19T10:04:39.205+0000,AB,abbandonata,undefined,undefined,previous conv
FmaadPOGBdKFYXbxDOTHfk,2018-12-19,2018-12-19T10:07:04.262+0000,2018-12-19T10:07:06.898+0000,AB,abbandonata,undefined,undefined,previous conv
GApzqyRtJHTGOdEoOQoKSZ,2018-12-19,2018-12-19T10:06:21.952+0000,2018-12-19T10:06:23.217+0000,AB,abbandonata,undefined,undefined,previous conv
GBs45imyGM4LhlCUxGtEKc,2018-12-19,2018-12-19T10:04:08.259+0000,2018-12-19T10:04:11.489+0000,AB,abbandonata,undefined,undefined,previous conv
21j4N9vh5jAJtG7qSjkDa2,2018-12-19,2018-12-19T10:05:27.887+0000,2018-12-19T10:05:29.128+0000,AB,abbandonata,undefined,undefined,previous conv
21j4N9vh5jAJtG7qSjkDa2,2018-12-19,2018-12-19T10:05:27.887+0000,2018-12-19T10:05:29.128+0000,AB,abbandonata,undefined,undefined,previous conv
3BrzSpxS6g3A5b9pHtU6mQ,2018-12-19,2018-12-19T10:00:46.011+0000,2018-12-19T10:00:50.725+0000,AB,abbandonata,undefined,undefined,previous conv
C2oJJD3kvuWBKRNfCqsjkl,2018-12-19,2018-12-19T10:07:50.791+0000,2018-12-19T10:07:53.466+0000,AB,abbandonata,undefined,undefined,previous conv
1Fj9IX6jUel4pxfWFsQXhZ,2018-12-19,2018-12-19T10:06:05.634+0000,2018-12-19T10:06:06.321+0000,AB,abbandonata,undefined,undefined,previous conv


### Step 1.2: FULL join between previous conversations and new (la left mantiene i record della prima, inner quelli comuni, la full tutti. Mi prendo conv vecchie, conv nuove, e con full join le ottengo tutte)

In [78]:
#display(previous_conversations.join(lastHourConversations, on= 'ConversationId', how='full'))

allConversations= previous_conversations.join(lastHourConversations, previous_conversations.ConversationId == lastHourConversations.NewConvId, how='full')

display(allConversations)

ConversationId,ConversationStartDate,ConversationStartTime,ConversationEndTime,ConversationStatusCode,ConversationStatus,ConversationRecognitionCode,ConversationRecognition,recordType,NewConvId,NewStartDate,NewStartTime,NewEndTime
7B0HJgE9ijNAHL1jPjChtq,2018-12-19,2018-12-19T09:59:58.615+0000,2018-12-19T10:00:01.362+0000,AB,abbandonata,undefined,undefined,previous conv,null,null,null,null
J2XpMj6qpqz1KvQbvZtk5q,2018-12-19,2018-12-19T10:07:54.357+0000,2018-12-19T10:07:57.099+0000,AB,abbandonata,undefined,undefined,previous conv,null,null,null,null
AV76YJrYqv8IRMzTI36tpW,2018-12-19,2018-12-19T10:06:11.685+0000,2018-12-19T10:06:14.778+0000,AB,abbandonata,undefined,undefined,previous conv,null,null,null,null
5c589WiBvSAGJRqJeYCipI,2018-12-19,2018-12-19T10:04:46.396+0000,2018-12-19T10:04:49.095+0000,AB,abbandonata,undefined,undefined,previous conv,null,null,null,null
5qvCp1zz8FEGRvqzzrI6QS,2018-12-19,2018-12-19T10:00:34.854+0000,2018-12-19T10:00:35.767+0000,CH,chiusa,NRIC,non riconosciuta,previous conv,null,null,null,null
6LRLPjJDLkQ85QplizAeFS,null,null,null,AB,abbandonata,undefined,undefined,previous conv,null,null,null,null
6LRLPjJDLkQ85QplizAeFS,null,null,null,CH,chiusa,undefined,undefined,previous conv,null,null,null,null
2Bi9prmuXhr3RTwBeOgfis,2018-12-19,2018-12-19T10:03:57.113+0000,2018-12-19T10:03:59.912+0000,AB,abbandonata,undefined,undefined,previous conv,null,null,null,null
9qn6DpEPlCmI89kgiaNs8v,2018-12-19,2018-12-19T10:01:44.270+0000,2018-12-19T10:01:47.835+0000,AB,abbandonata,undefined,undefined,previous conv,null,null,null,null
DwjuQDnNqmsDeFPwQDLqZX,2018-12-19,2018-12-19T10:05:22.290+0000,2018-12-19T10:05:25.040+0000,AB,abbandonata,undefined,undefined,previous conv,null,null,null,null


In [79]:
allConversations.count()  #131 che già avevo nella delta, più le 4 distinct dell'ultima ora

Out[ 182 ]: 135

#### Adding field recordType to indicate whether the conversation is new or not.

In [81]:
allConversations= allConversations.withColumn('recordType', when(allConversations['ConversationId'].isNull(), F.lit('new conv')).otherwise(allConversations['recordType']))
allConversations= allConversations.withColumn('ConversationId', when(allConversations['ConversationId'].isNull(), allConversations['NewConvId']).otherwise(allConversations['ConversationId']))
allConversations= allConversations.withColumn('ConversationStartDate', when(allConversations['ConversationStartDate'].isNull(), allConversations['NewStartDate']).otherwise(allConversations['ConversationStartDate']))
allConversations= allConversations.withColumn('ConversationStartTime', when(allConversations['ConversationStartTime'].isNull(), allConversations['NewStartTime']).otherwise(allConversations['ConversationStartTime']))
allConversations= allConversations.withColumn('ConversationEndTime', when(allConversations['ConversationEndTime'].isNull(), allConversations['NewEndTime']).otherwise(allConversations['ConversationEndTime']))

allConversations= allConversations.select([col for col in allConversations.columns if col not in ['NewStartDate', 'NewStartTime', 'NewEndTime']])
display(allConversations)

ConversationId,ConversationStartDate,ConversationStartTime,ConversationEndTime,ConversationStatusCode,ConversationStatus,ConversationRecognitionCode,ConversationRecognition,recordType,NewConvId
7B0HJgE9ijNAHL1jPjChtq,2018-12-19,2018-12-19T09:59:58.615+0000,2018-12-19T10:00:01.362+0000,AB,abbandonata,undefined,undefined,previous conv,null
J2XpMj6qpqz1KvQbvZtk5q,2018-12-19,2018-12-19T10:07:54.357+0000,2018-12-19T10:07:57.099+0000,AB,abbandonata,undefined,undefined,previous conv,null
AV76YJrYqv8IRMzTI36tpW,2018-12-19,2018-12-19T10:06:11.685+0000,2018-12-19T10:06:14.778+0000,AB,abbandonata,undefined,undefined,previous conv,null
5c589WiBvSAGJRqJeYCipI,2018-12-19,2018-12-19T10:04:46.396+0000,2018-12-19T10:04:49.095+0000,AB,abbandonata,undefined,undefined,previous conv,null
5qvCp1zz8FEGRvqzzrI6QS,2018-12-19,2018-12-19T10:00:34.854+0000,2018-12-19T10:00:35.767+0000,CH,chiusa,NRIC,non riconosciuta,previous conv,null
6LRLPjJDLkQ85QplizAeFS,null,null,null,AB,abbandonata,undefined,undefined,previous conv,null
6LRLPjJDLkQ85QplizAeFS,null,null,null,CH,chiusa,undefined,undefined,previous conv,null
2Bi9prmuXhr3RTwBeOgfis,2018-12-19,2018-12-19T10:03:57.113+0000,2018-12-19T10:03:59.912+0000,AB,abbandonata,undefined,undefined,previous conv,null
9qn6DpEPlCmI89kgiaNs8v,2018-12-19,2018-12-19T10:01:44.270+0000,2018-12-19T10:01:47.835+0000,AB,abbandonata,undefined,undefined,previous conv,null
DwjuQDnNqmsDeFPwQDLqZX,2018-12-19,2018-12-19T10:05:22.290+0000,2018-12-19T10:05:25.040+0000,AB,abbandonata,undefined,undefined,previous conv,null


In [82]:
"""new_Events2.createOrReplaceTempView("EventsFinal")
LastItemPerConv = spark.sql(" \
          SELECT ConversationId, max(eventTime) \
          FROM EventsFinal \
          GROUP BY ConversationId\
          ORDER BY max(eventTime)\
          ")
display(LastItemPerConv)"""
# Altro modo, penso più performante perchè non usa vista e spark.sql
LastItemPerConv = new_Events2.select('ConversationId', 'eventTime', 'Status').groupby('ConversationId').agg(F.max(new_Events2.eventTime)).select('*')
#display(new_Events2.select('ConversationId', 'eventTime', 'Status').groupby('ConversationId').agg({"eventTime": "max"}))  #F.max o :"max" così sono uguali
LastItemPerConv= LastItemPerConv.withColumnRenamed('max(eventTime)', 'eventTime')
display(LastItemPerConv)

ConversationId,eventTime
8mkbRBVLXnqB1DyyZ1pVcZ,2018-11-12T14:57:54.130+0000
777Qsjs3i8qJyBr95zABbk,2018-11-12T13:59:32.347+0000
2cpS4QCdLps7ycDeAxodOz,2018-11-12T14:07:45.439+0000
JEBXXIfay3m328mWQLvL0x,2018-11-12T14:55:34.613+0000


In [83]:
LastRecordPerConv= LastItemPerConv.join(new_Events2, on= ['ConversationId', 'eventTime'], how= 'left').withColumnRenamed('ConversationId', 'EventsConvId')
allConversationsUpdated = allConversations.join(LastRecordPerConv, (allConversations.ConversationId == LastRecordPerConv.EventsConvId), how= 'left')

display(LastRecordPerConv)

EventsConvId,eventTime,itemId,ClientAtHome,LUISentities,LUISintent,LUISscore,ProblemSolved,QNAanswer,QNAcategory,QNAscore,QNAsource,QNAuserMessage,TXTANALYTICSscore,activityId,channel,currentScenario,diagnosticResponse,dialogCloseType,endTimeAPICall,eventName,eventdate,isAuthenticated,isLeaf,mainScenario,modemAlignment,modemChanged,modemType,notUnderstoodMessage,reason,scenario,scenarioCalled,scenarioId,startTimeAPICall,telemetryId,text,ticket,ticketForm,timestamp,toolExec,toolResult,user,userId,userName,applicationInsight,businessUnit,type,apiTimeDiff,BotAnswerDuration,EventTimePrevious,DateUTCInsertRow,Environment,eventNameDecode,Status,Recognition,StatusCode,RecognitionCode,HourFromLastEvent,EventHourMinute
2cpS4QCdLps7ycDeAxodOz,2018-11-12T14:07:45.439+0000,5acf0b77-e684-11e8-8c74-1df3eaf4721b,,,,,,,,,,,null,2cpS4QCdLps7ycDeAxodOz|0000116,pat,WO_Problemi_Navigazione,,,null,MBFEvent.BotMessage,2018-11-12,false,false,WO_Inizio,,,,,,,,adf415c5-3843-4587-9e04-e28ebef74e63,null,,Qual è il tuo problema?,,,2018-11-12T14:07:45.439+0000,,,,dev-timva-wo-botservice@-164JqvoUec,dev-timva-wo-botservice,appinsights-wo,wo,new,null,855.895,2018-11-12T14:07:44.583+0000,2018-12-19T17:52:24.577+0000,DEV,BotMessage,abbandonata,undefined,AB,undefined,890.74,1407
JEBXXIfay3m328mWQLvL0x,2018-11-12T14:55:34.613+0000,0d572e21-e68b-11e8-a351-0f405d5c52f9,,,,,,,,,,,null,0b7dab48-7c1e-415b-8093-6b11b98095bc,pat,Feedback Dialog,,,null,MBFEvent.BotMessage,2018-11-12,false,false,WO_Inizio,,,,,,,,54188222-5a83-424b-baed-1e1438772a90,null,,Sono stato utile?,,,2018-11-12T14:55:34.595+0000,,,,dev-timva-wo-botservice@-164JqvoUec,dev-timva-wo-botservice,appinsights-wo,wo,new,null,null,null,2018-12-19T17:52:24.577+0000,DEV,BotMessage,abbandonata,undefined,AB,undefined,889.95,1455
8mkbRBVLXnqB1DyyZ1pVcZ,2018-11-12T14:57:54.130+0000,55cd7162-e68b-11e8-bf7a-5fba8834ae28,,,,,,,,,,,null,8mkbRBVLXnqB1DyyZ1pVcZ|0000723,pat,Welcome,,,null,MBFEvent.BotMessage,2018-11-12,true,true,Reset Dialog,,,,,,,,cea8d6a0-d975-4129-b098-9892bb739e29,null,,"Ciao, sono l'assistente virtuale dell'assistenza tecnica. Come posso aiutarti?",,,2018-11-12T14:57:54.121+0000,,,FE054A8741794EEA06252DD6DB61F8DA,dev-timva-wo-botservice@-164JqvoUec,dev-timva-wo-botservice,appinsights-wo,wo,new,null,null,null,2018-12-19T17:52:24.577+0000,DEV,BotMessage,chiusa,undefined,CH,undefined,889.91,1457
777Qsjs3i8qJyBr95zABbk,2018-11-12T13:59:32.347+0000,3b79ea7c-e683-11e8-8c74-1df3eaf4721b,,,,,,,,,,,null,777Qsjs3i8qJyBr95zABbk|0000097,pat,WO_Reset_Porta_Voip,,,null,MBFEvent.BotMessage,2018-11-12,false,false,WO_Reset_Porta_Voip,,,,,,,,6903fb36-c709-4669-82d0-1a69c9c65fae,null,,"Stiamo effettuando un reset in centrale. Spegni e riaccendi il modem, attendi che le spie siano di nuovo accese. Dimmi quando hai finito.",,,2018-11-12T13:59:32.347+0000,,,,dev-timva-wo-botservice@-164JqvoUec,dev-timva-wo-botservice,appinsights-wo,wo,new,null,null,null,2018-12-19T17:52:24.577+0000,DEV,BotMessage,abbandonata,undefined,AB,undefined,890.88,1359
777Qsjs3i8qJyBr95zABbk,2018-11-12T13:59:32.347+0000,3b79ea7b-e683-11e8-8c74-1df3eaf4721b,,,,,,,,,,,null,777Qsjs3i8qJyBr95zABbk|0000097,pat,,,,2018-11-12T13:59:32.347+0000,MBFEvent.APICall,2018-11-12,,,WO_Reset_Porta_Voip,,,,,,,,6903fb36-c709-4669-82d0-1a69c9c65fae,2018-11-12T13:59:32.139+0000,,,,,2018-11-12T13:59:28.942+0000,,,,5f9d9ab4e3fcaecdabbed1228ffaacab7f942b91,,appinsights-wo,wo,new,207.536,null,null,2018-12-19T17:52:24.577+0000,DEV,APICall,abbandonata,undefined,AB,undefined,890.88,1359


In [84]:
#allConversationsUpdated.persist()

In [85]:
allConversationsUpdated.columns

Out[ 186 ]: 
['ConversationId',
 'ConversationStartDate',
 'ConversationStartTime',
 'ConversationEndTime',
 'ConversationStatusCode',
 'ConversationStatus',
 'ConversationRecognitionCode',
 'ConversationRecognition',
 'recordType',
 'NewConvId',
 'EventsConvId',
 'eventTime',
 'itemId',
 'ClientAtHome',
 'LUISentities',
 'LUISintent',
 'LUISscore',
 'ProblemSolved',
 'QNAanswer',
 'QNAcategory',
 'QNAscore',
 'QNAsource',
 'QNAuserMessage',
 'TXTANALYTICSscore',
 'activityId',
 'channel',
 'currentScenario',
 'diagnosticResponse',
 'dialogCloseType',
 'endTimeAPICall',
 'eventName',
 'eventdate',
 'isAuthenticated',
 'isLeaf',
 'mainScenario',
 'modemAlignment',
 'modemChanged',
 'modemType',
 'notUnderstoodMessage',
 'reason',
 'scenario',
 'scenarioCalled',
 'scenarioId',
 'startTimeAPICall',
 'telemetryId',
 'text',
 'ticket',
 'ticketForm',
 'timestamp',
 'toolExec',
 'toolResult',
 'user',
 'userId',
 'userName',
 'applicationInsight',
 'businessUnit',
 'type',
 'apiTimeDiff',
 'BotAnswerDuration',
 'EventTimePrevious',
 'DateUTCInsertRow',
 'Environment',
 'eventNameDecode',
 'Status',
 'Recognition',
 'StatusCode',
 'RecognitionCode',
 'HourFromLastEvent',
 'EventHourMinute']

In [86]:
allConversationsUpdated= allConversationsUpdated.select([col for col in allConversations.columns + ['Status', 'Recognition', 'StatusCode', 'RecognitionCode','DateUTCInsertRow'] ]).withColumn('ConversationStatus', when(allConversationsUpdated.Status.isNotNull(), allConversationsUpdated.Status).otherwise(allConversationsUpdated.ConversationStatus))

allConversationsUpdated= allConversationsUpdated.withColumn('ConversationRecognitionCode', when(allConversationsUpdated.RecognitionCode.isNotNull(), allConversationsUpdated.RecognitionCode).otherwise(allConversationsUpdated.ConversationRecognitionCode)).withColumn('ConversationRecognition', when(allConversationsUpdated.Recognition.isNotNull(), allConversationsUpdated.Recognition).otherwise(allConversationsUpdated.ConversationRecognition)).withColumn('ConversationStatusCode', when(allConversationsUpdated.StatusCode.isNotNull(), allConversationsUpdated.StatusCode).otherwise(allConversationsUpdated.ConversationStatusCode))

allConversationsUpdated= allConversationsUpdated.select([col for col in allConversations.columns + ['DateUTCInsertRow'] if col!= 'NewConvId'])
display(allConversationsUpdated)

# DateUTCInsertRow è valorizzato solo per quelle nuove, solamente perchè per quelle vecchie il codice era diverso...

ConversationId,ConversationStartDate,ConversationStartTime,ConversationEndTime,ConversationStatusCode,ConversationStatus,ConversationRecognitionCode,ConversationRecognition,recordType,DateUTCInsertRow
7B0HJgE9ijNAHL1jPjChtq,2018-12-19,2018-12-19T09:59:58.615+0000,2018-12-19T10:00:01.362+0000,AB,abbandonata,undefined,undefined,previous conv,null
J2XpMj6qpqz1KvQbvZtk5q,2018-12-19,2018-12-19T10:07:54.357+0000,2018-12-19T10:07:57.099+0000,AB,abbandonata,undefined,undefined,previous conv,null
AV76YJrYqv8IRMzTI36tpW,2018-12-19,2018-12-19T10:06:11.685+0000,2018-12-19T10:06:14.778+0000,AB,abbandonata,undefined,undefined,previous conv,null
5c589WiBvSAGJRqJeYCipI,2018-12-19,2018-12-19T10:04:46.396+0000,2018-12-19T10:04:49.095+0000,AB,abbandonata,undefined,undefined,previous conv,null
5qvCp1zz8FEGRvqzzrI6QS,2018-12-19,2018-12-19T10:00:34.854+0000,2018-12-19T10:00:35.767+0000,CH,chiusa,NRIC,non riconosciuta,previous conv,null
6LRLPjJDLkQ85QplizAeFS,null,null,null,AB,abbandonata,undefined,undefined,previous conv,null
6LRLPjJDLkQ85QplizAeFS,null,null,null,CH,chiusa,undefined,undefined,previous conv,null
2Bi9prmuXhr3RTwBeOgfis,2018-12-19,2018-12-19T10:03:57.113+0000,2018-12-19T10:03:59.912+0000,AB,abbandonata,undefined,undefined,previous conv,null
9qn6DpEPlCmI89kgiaNs8v,2018-12-19,2018-12-19T10:01:44.270+0000,2018-12-19T10:01:47.835+0000,AB,abbandonata,undefined,undefined,previous conv,null
DwjuQDnNqmsDeFPwQDLqZX,2018-12-19,2018-12-19T10:05:22.290+0000,2018-12-19T10:05:25.040+0000,AB,abbandonata,undefined,undefined,previous conv,null


In [87]:
##########################################################################################

### Split: I separate Hot and Cold Conversations

In [89]:
hotConversations= allConversationsUpdated.filter((allConversationsUpdated.ConversationStatus!= 'abbandonata') & (allConversationsUpdated.ConversationStatus!='chiusa'))
coldConversations= allConversationsUpdated.filter((allConversationsUpdated.ConversationStatus=='abbandonata') | (allConversationsUpdated.ConversationStatus=='chiusa'))
                                              
display(coldConversations)  #sono 135

ConversationId,ConversationStartDate,ConversationStartTime,ConversationEndTime,ConversationStatusCode,ConversationStatus,ConversationRecognitionCode,ConversationRecognition,recordType,DateUTCInsertRow
7B0HJgE9ijNAHL1jPjChtq,2018-12-19,2018-12-19T09:59:58.615+0000,2018-12-19T10:00:01.362+0000,AB,abbandonata,undefined,undefined,previous conv,null
J2XpMj6qpqz1KvQbvZtk5q,2018-12-19,2018-12-19T10:07:54.357+0000,2018-12-19T10:07:57.099+0000,AB,abbandonata,undefined,undefined,previous conv,null
AV76YJrYqv8IRMzTI36tpW,2018-12-19,2018-12-19T10:06:11.685+0000,2018-12-19T10:06:14.778+0000,AB,abbandonata,undefined,undefined,previous conv,null
5c589WiBvSAGJRqJeYCipI,2018-12-19,2018-12-19T10:04:46.396+0000,2018-12-19T10:04:49.095+0000,AB,abbandonata,undefined,undefined,previous conv,null
5qvCp1zz8FEGRvqzzrI6QS,2018-12-19,2018-12-19T10:00:34.854+0000,2018-12-19T10:00:35.767+0000,CH,chiusa,NRIC,non riconosciuta,previous conv,null
6LRLPjJDLkQ85QplizAeFS,null,null,null,AB,abbandonata,undefined,undefined,previous conv,null
6LRLPjJDLkQ85QplizAeFS,null,null,null,CH,chiusa,undefined,undefined,previous conv,null
2Bi9prmuXhr3RTwBeOgfis,2018-12-19,2018-12-19T10:03:57.113+0000,2018-12-19T10:03:59.912+0000,AB,abbandonata,undefined,undefined,previous conv,null
9qn6DpEPlCmI89kgiaNs8v,2018-12-19,2018-12-19T10:01:44.270+0000,2018-12-19T10:01:47.835+0000,AB,abbandonata,undefined,undefined,previous conv,null
DwjuQDnNqmsDeFPwQDLqZX,2018-12-19,2018-12-19T10:05:22.290+0000,2018-12-19T10:05:25.040+0000,AB,abbandonata,undefined,undefined,previous conv,null


In [90]:
hotConversations_noUTC= hotConversations.select([col for col in hotConversations.columns if col != 'DateUTCInsertRow'])
coldConversations_noUTC= coldConversations.select([col for col in coldConversations.columns if col != 'DateUTCInsertRow'])

In [91]:
%sql
SELECT count(*)
FROM DEV.Conversations

count(1)
131


In [ ]:
"""IN TEORIA ADESSO SOVRASCRIVEREI COLD CONVERSATIONS (135 RECORDS) NELLA DELTA, MA MENTRE FACEVO QUESTI RUN STAVA RUN
STAVA RUNNANDO ANCHE LA SCHEDULAZIONE CHE CANCELLA FILES E RECORDS DA Conversations E DA Events, per cui alla fine di questa
parte 1 invece di avere 135 records su conversations, ne avrò solamente 5. Parte 2 non inizia quindi con 135 records ma 5."""

## Saving Tables

In [92]:
#write in delta table in OVERWRITE   DEV.Conversation
#hotConversations.select([col for col in hotConversations.columns if col!= 'DateUTCInsertRow' ]).write.format("delta").mode("overwrite").save("/delta/"+environment+"/Conversations/")
hotConversations_noUTC.write.format("delta").mode("overwrite").save("/delta/"+environment+"/Conversations/")

In [93]:
#spark.sql select per far matchare campi con quelli del DataWharehouse
hotConversations.createOrReplaceTempView("hotConversations")

sqlConversationsHot = spark.sql("""\
SELECT ConversationId
, ConversationStartDate
, ConversationStartTime
, ConversationEndTime
, ConversationStatusCode
, ConversationStatus
, ConversationRecognitionCode
, ConversationRecognition
, 0 as Count_BotMessage
, 0 as Count_UserMessage
, 0 as Count_NotUnderstood
, 0 as Count_Understood
, 0 as Count_LeafEvent
, 0 as Count_Ticket
, 0 as Count_CloseEvent
, 0 as Count_CloseByBot
, '' as DateUTCInsertRow               -- 11/12/2018
, '' as DateUTCInsertRow_Europe_Rome   -- 11/12/2018
FROM hotConversations
""")

#display(sqlConversationsHot)

In [94]:
#spark.sql select per far matchare campi con quelli del DataWharehouse
coldConversations.createOrReplaceTempView("coldConversations")

sqlConversationsCold = spark.sql("""\
SELECT ConversationId
, ConversationStartDate
, ConversationStartTime
, ConversationEndTime
, ConversationStatusCode
, ConversationStatus
, ConversationRecognitionCode
, ConversationRecognition
, 0 as Count_BotMessage
, 0 as Count_UserMessage
, 0 as Count_NotUnderstood
, 0 as Count_Understood
, 0 as Count_LeafEvent
, 0 as Count_Ticket
, 0 as Count_CloseEvent
, 0 as Count_CloseByBot
, DateUTCInsertRow as DateUTCInsertRow              -- 11/12/2018
, '' as DateUTCInsertRow_Europe_Rome  -- 11/12/2018
FROM coldConversations
""")

display(sqlConversationsCold)

ConversationId,ConversationStartDate,ConversationStartTime,ConversationEndTime,ConversationStatusCode,ConversationStatus,ConversationRecognitionCode,ConversationRecognition,Count_BotMessage,Count_UserMessage,Count_NotUnderstood,Count_Understood,Count_LeafEvent,Count_Ticket,Count_CloseEvent,Count_CloseByBot,DateUTCInsertRow,DateUTCInsertRow_Europe_Rome
7B0HJgE9ijNAHL1jPjChtq,2018-12-19,2018-12-19T09:59:58.615+0000,2018-12-19T10:00:01.362+0000,AB,abbandonata,undefined,undefined,0,0,0,0,0,0,0,0,2018-12-19T12:21:18.084+0000,
J2XpMj6qpqz1KvQbvZtk5q,2018-12-19,2018-12-19T10:07:54.357+0000,2018-12-19T10:07:57.099+0000,AB,abbandonata,undefined,undefined,0,0,0,0,0,0,0,0,2018-12-19T12:21:18.084+0000,
AV76YJrYqv8IRMzTI36tpW,2018-12-19,2018-12-19T10:06:11.685+0000,2018-12-19T10:06:14.778+0000,AB,abbandonata,undefined,undefined,0,0,0,0,0,0,0,0,2018-12-19T12:21:18.084+0000,
5c589WiBvSAGJRqJeYCipI,2018-12-19,2018-12-19T10:04:46.396+0000,2018-12-19T10:04:49.095+0000,AB,abbandonata,undefined,undefined,0,0,0,0,0,0,0,0,2018-12-19T12:21:18.084+0000,
5qvCp1zz8FEGRvqzzrI6QS,2018-12-19,2018-12-19T10:00:34.854+0000,2018-12-19T10:00:35.767+0000,CH,chiusa,NRIC,non riconosciuta,0,0,0,0,0,0,0,0,2018-12-19T12:21:18.084+0000,
6LRLPjJDLkQ85QplizAeFS,null,null,null,AB,abbandonata,undefined,undefined,0,0,0,0,0,0,0,0,null,
6LRLPjJDLkQ85QplizAeFS,null,null,null,CH,chiusa,undefined,undefined,0,0,0,0,0,0,0,0,null,
2Bi9prmuXhr3RTwBeOgfis,2018-12-19,2018-12-19T10:03:57.113+0000,2018-12-19T10:03:59.912+0000,AB,abbandonata,undefined,undefined,0,0,0,0,0,0,0,0,2018-12-19T12:21:18.084+0000,
9qn6DpEPlCmI89kgiaNs8v,2018-12-19,2018-12-19T10:01:44.270+0000,2018-12-19T10:01:47.835+0000,AB,abbandonata,undefined,undefined,0,0,0,0,0,0,0,0,2018-12-19T12:21:18.084+0000,
DwjuQDnNqmsDeFPwQDLqZX,2018-12-19,2018-12-19T10:05:22.290+0000,2018-12-19T10:05:25.040+0000,AB,abbandonata,undefined,undefined,0,0,0,0,0,0,0,0,2018-12-19T12:21:18.084+0000,


In [95]:
%sql

SET spark.databricks.queryWatchdog.enabled=false

In [96]:
#WRITE IN DW in table [ConversationCold]  (in APPEND)
sqlConversationsCold.write.option("maxStrLength",4000).format("com.databricks.spark.sqldw").option("url", connStringDWH).option("dbtable", "ConversationCold").option("forward_spark_azure_storage_credentials","True").mode("append").option("tempdir", tempdatabricks_wo).save()

In [97]:
#WRITE IN DW in table [ConversationHot]  (in OVERWRITE)
sqlConversationsHot.write.option("maxStrLength",4000).format("com.databricks.spark.sqldw").option("url", connStringDWH).option("dbtable", "ConversationHot").option("forward_spark_azure_storage_credentials","True").mode("overwrite").option("tempdir", tempdatabricks_wo).save()

In [98]:
"""%sql
DESCRIBE FORMATTED DEV.Events  --per sapere schema """
# PORTO AVANTI SOLAMENTE CONVERSATIONHOT --> VADO IN JOIN CON TABELLA EVENTS PER PRENDERE SOLO GLI EVENTI DELLE CONVERSAZIONI HOT. POI SOVRASCRIVO QUESTI NELLA DELTA

eventsHot= new_Events2.join(hotConversations_noUTC, on=['ConversationId'], how= 'inner')
eventsHot= eventsHot.select([col for col in new_Events2.columns if col not in hotConversations_noUTC.drop('ConversationId').columns])
display(eventsHot)

itemId,ClientAtHome,LUISentities,LUISintent,LUISscore,ProblemSolved,QNAanswer,QNAcategory,QNAscore,QNAsource,QNAuserMessage,TXTANALYTICSscore,activityId,channel,conversationId,currentScenario,diagnosticResponse,dialogCloseType,endTimeAPICall,eventName,eventTime,eventdate,isAuthenticated,isLeaf,mainScenario,modemAlignment,modemChanged,modemType,notUnderstoodMessage,reason,scenario,scenarioCalled,scenarioId,startTimeAPICall,telemetryId,text,ticket,ticketForm,timestamp,toolExec,toolResult,user,userId,userName,applicationInsight,businessUnit,type,apiTimeDiff,BotAnswerDuration,EventTimePrevious,DateUTCInsertRow,Environment,eventNameDecode,Status,Recognition,StatusCode,RecognitionCode,HourFromLastEvent,EventHourMinute


In [99]:
#display(coldConversations_noUTC.distinct())   #display vecchio

ConversationId,ConversationStartDate,ConversationStartTime,ConversationEndTime,ConversationStatusCode,ConversationStatus,ConversationRecognitionCode,ConversationRecognition,recordType
6LRLPjJDLkQ85QplizAeFS,null,null,null,CH,chiusa,undefined,undefined,new conv
6LRLPjJDLkQ85QplizAeFS,null,null,null,AB,abbandonata,undefined,undefined,new conv


In [100]:
#coldConversations_noUTC.select('ConversationId').distinct().count()
#display(coldConversations_noUTC.select('ConversationId').distinct())   #display vecchio

ConversationId
6LRLPjJDLkQ85QplizAeFS


In [101]:
#analogo a eventsHot
#eventsCold= new_Events2.join(coldConversations_noUTC, on= ['ConversationId'], how= 'inner')
eventsCold= new_Events2.join(coldConversations_noUTC, on=['ConversationId'], how= 'inner')
eventsCold= eventsCold.select([col for col in new_Events2.columns if col not in coldConversations_noUTC.drop('ConversationId').columns])
#display(eventsCold)

In [102]:
eventsCold.count()  #1021 records, che sono tutti quelli che sovrascrivo sulla delta

Out[ 193 ]: 1021

In [103]:
# eventsHot.count()  --> è zero

In [105]:
eventsCold.write.format("delta").mode("overwrite").save("/delta/"+environment+"/Events_Federico/")  #Siccome eventsHot è vuota (è Hot se è negli ultimi 20-30 minuti...ma sto runnando orari piu vecchi quindi saranno tutte cold..), sovrascrivo sulla delta eventsCold, così la riempio. Sennò rimane con zero records...

In [106]:
%sql
--SELECT count(*)
--FROM DEV.Events

count(1)
1021


In [107]:
%sql
SELECT count(*)   
FROM DEV.Events_Federico   -- Ho fatto una copia della Delta Table

count(1)
1021


In [108]:
#write in delta table (in OVERWRITE)
eventsHot.write.format("delta").mode("overwrite").save("/delta/"+environment+"/Events/") 

In [109]:
#spark.sql select per far matchare campi con quelli del DataWharehouse
eventsCold.createOrReplaceTempView("eventsCold")

sqlEventsCold = spark.sql("""
SELECT
  '' as ID
, CAST(eventTime as timestamp) as EventTime
, year(eventTime) * 10000 + month(eventTime) * 100 + day(eventTime) as EventDateNum
, hour(eventTime) as EventHourNum
, minute(eventTime) as EventMinuteNum
, second(eventTime) as EventSecondNum
, EventName as EventName
, '' as internalID
, '' as ClientIP
, '' as Continent
, '' as Country
, '' as Province
, '' as City
, LEFT(Text,4000) as Text
, ConversationId as ConversationId
, Channel
, Timestamp
, UserName
, UserId
, LUISintent
, LUISscore
, MainScenario
, Reason
, Scenario
, CurrentScenario
, QNAscore as QNAscore
, QNAanswer as QNAanswer
, QNAuserMessage as QNAuserMessage
, QNAsource as QNAsource
, QNAcategory as QNAcategory
, ItemId as ItemId
, ActivityId as ActivityId
, BotAnswerDuration as DurationAnswerBot
, CAST(startTimeAPICall as timestamp) as startTimeAPICall
, CAST(endTimeAPICall as timestamp) as endTimeAPICall
, apiTimeDiff as DurationAnswerAPI
, '' as modem
, '' as linea
, ticket as ticket
, ticketForm as ticketForm
, ClientAtHome as ClientAtHome
, modemType as modemType
, diagnosticResponse as diagnosticResponse
, modemAlignment as modemAlignment
, toolExec as toolExec
, toolResult as toolResult
, TXTANALYTICSscore as TXTANALYTICSscore    -- 03/12/2018 converted from string to float
, scenarioid as scenarioid
, scenariocalled as scenariocalled
, isLeaf as isLeafText
, DateUTCInsertRow as DateUTCInsertRow
, modemChanged as modemChanged
, dialogCloseType as dialogCloseType
, EventTimePrevious as EventTimePrevious
, notUnderstoodMessage as notUnderstoodMessage
, applicationInsight as applicationInsight
, businessUnit as businessUnit
, problemSolved as ProblemSolved                      -- before "ProblemSolved", after "problemSolved"
, user as user
, isAuthenticated as isAuthenticated
, '' as Status -- added on 2018-11-23
, '' as Recognition -- added on 2018-11-23
, '' as StatusCode -- added on 2018-11-23
, '' as RecognitionCode -- added on 2018-11-23
, 0 as HourFromLastEvent -- added on 2018-11-23, manually set to zero because field type in DW is number, waiting valorization from notebook
, 0 as EventHourMinute -- added on 2018-11-23, manually set to zero because field type in DW is number, waiting valorization from notebook
, Environment as Environment
, eventNameDecode as eventNameDecode
FROM eventsCold
""")

In [110]:
#WRITE IN DW in table [eventsCold]  (in APPEND)
sqlEventsCold.write.option("maxStrLength",4000).format("com.databricks.spark.sqldw").option("url", connStringDWH).option("dbtable", "EventCold").option("forward_spark_azure_storage_credentials","True").mode("append").option("tempdir", tempdatabricks_wo).save()

In [111]:
#PREPARING RESULTSET TO WRITE INTO DW
# ATTENTION: NUMBER OF COLUMNS MUST MATCH WITH OUTPUT AND IN THE SAME ORDER OF DB TABLE STRUCTURE


# Create or Replace TempView, uncomment if sql manipulation must be used
eventsHot.createOrReplaceTempView("eventsHot")

sqlEventsHot = spark.sql("""
SELECT
  '' as ID
, CAST(eventTime as timestamp) as EventTime
, year(eventTime) * 10000 + month(eventTime) * 100 + day(eventTime) as EventDateNum
, hour(eventTime) as EventHourNum
, minute(eventTime) as EventMinuteNum
, second(eventTime) as EventSecondNum
, EventName as EventName
, '' as internalID
, '' as ClientIP
, '' as Continent
, '' as Country
, '' as Province
, '' as City
, LEFT(Text,4000) as Text
, ConversationId as ConversationId
, Channel
, Timestamp
, UserName
, UserId
, LUISintent
, LUISscore
, MainScenario
, Reason
, Scenario
, CurrentScenario
, QNAscore as QNAscore
, QNAanswer as QNAanswer
, QNAuserMessage as QNAuserMessage
, QNAsource as QNAsource
, QNAcategory as QNAcategory
, ItemId as ItemId
, ActivityId as ActivityId
, BotAnswerDuration as DurationAnswerBot
, CAST(startTimeAPICall as timestamp) as startTimeAPICall
, CAST(endTimeAPICall as timestamp) as endTimeAPICall
, apiTimeDiff as DurationAnswerAPI
, '' as modem
, '' as linea
, ticket as ticket
, ticketForm as ticketForm
, ClientAtHome as ClientAtHome
, modemType as modemType
, diagnosticResponse as diagnosticResponse
, modemAlignment as modemAlignment
, toolExec as toolExec
, toolResult as toolResult
, TXTANALYTICSscore as TXTANALYTICSscore    -- 03/12/2018 converted from string to float
, scenarioid as scenarioid
, scenariocalled as scenariocalled
, isLeaf as isLeafText
, DateUTCInsertRow as DateUTCInsertRow
, modemChanged as modemChanged
, dialogCloseType as dialogCloseType
, EventTimePrevious as EventTimePrevious
, notUnderstoodMessage as notUnderstoodMessage
, applicationInsight as applicationInsight
, businessUnit as businessUnit
, problemSolved as ProblemSolved                      -- before "ProblemSolved", after "problemSolved"
, user as user
, isAuthenticated as isAuthenticated
, '' as Status -- added on 2018-11-23
, '' as Recognition -- added on 2018-11-23
, '' as StatusCode -- added on 2018-11-23
, '' as RecognitionCode -- added on 2018-11-23
, 0 as HourFromLastEvent -- added on 2018-11-23, manually set to zero because field type in DW is number, waiting valorization from notebook
, 0 as EventHourMinute -- added on 2018-11-23, manually set to zero because field type in DW is number, waiting valorization from notebook
, Environment as Environment
, eventNameDecode as eventNameDecode
FROM eventsHot
""")

In [112]:
###WRITE IN DW in table [EventHot] (NB: now write option is "Append" but when will have Hot and Cold logic it will be "Overwrite" in Hot Table and "Append" in Cold Table)
#WRITE IN DW in table [EventHot]  (in OVERWRITE)
sqlEventsHot.write.option("maxStrLength",4000).format("com.databricks.spark.sqldw").option("url", connStringDWH).option("dbtable", "EventHot").option("forward_spark_azure_storage_credentials","True").mode("overwrite").option("tempdir", tempdatabricks_wo).save()

In [113]:
#allConversationsUpdated.unpersist() #dbutils.fs.rm("/delta/DEV/Events/", True)

In [114]:
#command to manage "user authenticated"
new_Events2.filter(new_Events2.type == 'new').createOrReplaceTempView("finalTempTable")

#select for "user authenticated"
sqlQueryResult_User = spark.sql("""
SELECT
ConversationId as ConversationId
, user as user
-- DateUTCInsertRow as DateUTCInsertRow,    --to be added
FROM finalTempTable
WHERE
isAuthenticated = 'true' and (user is not null or user <> '')
GROUP BY
ConversationId, user
""")

#WRITE IN DW in table [DIM_User]
sqlQueryResult_User.write.option("maxStrLength",4000).format("com.databricks.spark.sqldw").option("url", connStringDWH).option("dbtable", "DIM_User").option("forward_spark_azure_storage_credentials","True").mode("append").option("tempdir", tempdatabricks_wo).save()

In [115]:
"""%sql
OPTIMIZE QA.Events --Dalta Table "Events" is partitioned by "eventDate"
WHERE EventDate >= current_timestamp() - INTERVAL 1 day
ZORDER BY (EventName, ConversationId)"""

Out[ 73 ]: '%sql\nOPTIMIZE QA.Events --Dalta Table "Events" is partitioned by "eventDate"\nWHERE EventDate >= current_timestamp() - INTERVAL 1 day\nZORDER BY (EventName, ConversationId)'

In [116]:
##Andrea 26/11/2018: Optimize disabled because now is scheduled in a specific notebook wich runs daily during the night
#optimize = spark.sql("OPTIMIZE {}.Events WHERE EventDate >= current_timestamp() - INTERVAL 1 day ZORDER BY (EventName, ConversationId)".format(environment))